# Young Diagrams and Tableau

## Diagrams

In [1]:
from diagram import Diagram
from tableau import Tableau

In [2]:
ls = [1]
ls[1:]

[]

In [3]:
import sympy as sp
import gravipy as gp
import numpy as np
import itertools
from cycle import Cycle
import copy as cp
import fractions as fr

class CycleSum():
    
    def __init__(self, cycle_list, prefactors = [], str_prefactors = [], normalise = False):
    
        cyc_lis = []
        prefacs = []
        self.normalised = False
        for cycle_ind in range(len(cycle_list)):
            
            if (len(cycle_list) == len(prefactors)):
                prefacs.append(prefactors[cycle_ind])
                if prefactors[cycle_ind] != 0:
                    if len(cycle_list) == len(str_prefactors):
                        cyc_lis.append({'cycle': cycle_list[cycle_ind], 'prefactor': fr.Fraction(prefactors[cycle_ind]),
                                    'prefactor_str': str_prefactors[cycle_ind]})
                    else:
                        cyc_lis.append({'cycle': cycle_list[cycle_ind], 'prefactor': fr.Fraction(prefactors[cycle_ind]),
                                    'prefactor_str': str(prefactors[cycle_ind])})
            else:
                cyc_lis.append({'cycle': cycle_list[cycle_ind], 'prefactor': fr.Fraction(1),
                                'prefactor_str': str(1)})
                prefacs.append(1)
                
        self.cycle_sum = cyc_lis
        if normalise:
            self.cycle_sum = self.normalise().get_sum()
        
        self.prefactors = prefacs
        
        
    def is_normalised(self):
        
        return self.normalised
    
        
    def get_operator(self):
        
        return self
        
        
    def get_sum(self):
        
        return self.cycle_sum
    
    
    def get_total_domain(self):
    
        all_cycs = self.get('cycle')
        doms = []
        for cyc in all_cycs:
            dom = cyc.get('domain')
            for ent in dom:
                doms.append(ent)
        
        return list(set(doms))
    
    
    def get_overall_factor(self):
        
        overall_fac = 0
        
        prefs = self.get('prefactor')
        denoms = [pref.denominator for pref in prefs]
        numers = [pref.numerator for pref in prefs]  
        
        if (len(denoms) > 0) and (len(numers) > 0):
            denom_fac = np.gcd.reduce(denoms)
            numer_fac = np.gcd.reduce(numers)

            overall_fac = fr.Fraction(numer_fac,denom_fac)     
            
        return overall_fac
        
        
    def get(self, prpty):
        
        if prpty is 'domain':
            return self.get_total_domain()
        else:
            return [cyc.get(prpty, None) for cyc in self.cycle_sum]
    
    
    def set_cycle_sum(self, val):
        
        self.cycle_sum = val
        
    def set_normalised(self, val):
        
        self.normalised = val
    
    
    def sum_with(self, cycles, factor = 1):
        
        added = cp.deepcopy(self).get('cycle')
        prefactors = cp.deepcopy(self).get('prefactor')

        added += cycles.get('cycle')
        prefactors += [fr.Fraction(factor)*fr.Fraction(cy) for cy in cycles.get('prefactor')]

        added_cycs = CycleSum(added, prefactors = prefactors).simplify()
        
        return added_cycs
    
    
    def sum_with_list(self, cycles, factor = 1):
        
        added = cp.deepcopy(self)
        
        for cyc in cycles:
            
            added = added.sum_with(cyc, factor = factor)
            
        return added

    
    def multiply_by_constant(self, const):
        
        cycles = self.get('cycle')
        prefacs = self.get('prefactor')
        
        prefactors = [fr.Fraction(const)*fr.Fraction(pref) for pref in prefacs]
        
        return CycleSum(cycles, prefactors = prefactors)
    
    
    def reverse(self):

        for cyc_ind in range(len(self.cycle_sum)):
            self.cycle_sum[cyc_ind]['cycle'] = self.cycle_sum[cyc_ind]['cycle'].reverse()            
        return self
    
    
        
    def normalise(self):
        
        rtn = self
        
        if not self.normalised:
            if self.is_idempotent():
    
                norm_fac = self.get_normalisation_factor()
                if norm_fac is not None:
                    rtn = self.multiply_by_constant(norm_fac)
                    rtn.set_normalised(True)
                else:
                    print('Error: operator cannot be normalised: operator is vanishing.')
            else:
                print('Error: operator cannot be normalised: is not idempotent')
                
        return rtn
    
    
    def get_normalisation_factor(self):
        
        norm_fac = None
        if self.is_idempotent():
    
            squared = self.act_on(self)
            norm_fac = fr.Fraction(1)
        
            if len(self.cycle_sum) > 0:
                first_term = self.cycle_sum[0]['cycle']
                first_pref = self.cycle_sum[0]['prefactor']
                sq_pref = None
                ind = 0
                while sq_pref is None:
                    sq_term = squared.get_sum()[ind]['cycle']

                    if sq_term.equivalent(first_term):
                        sq_pref = squared.get_sum()[ind]['prefactor']
                        
                        norm_fac = fr.Fraction(first_pref,sq_pref)
                    ind += 1
            else:
                norm_fac = None
                
        return norm_fac
        

    
    
    def is_idempotent(self):
        
        idem = False
        slf = cp.deepcopy(self)
        squared = slf.act_on(slf)
        
        if len(squared.get_sum()) == len(slf.get_sum()):
            if squared.is_equivalent_to(slf, up_to_factor = True):
                idem = True
        
        return idem
    
    
    def is_equivalent_to(self, cycsum, up_to_factor = False):
        
        equiv = False
        cyc_sum = cycsum
        slf = cp.deepcopy(self)

        if up_to_factor:
            print('fac')
            pref_slf = slf.get_overall_factor()
            print('myfac', pref_slf)
            pref_cyc = cyc_sum.get_overall_factor()
            print('cycpref', pref_cyc)
            slf = slf.multiply_by_constant(pref_cyc)
            cyc_sum = cyc_sum.multiply_by_constant(pref_slf)
            
        summed = slf.sum_with(cyc_sum, factor = -1)
        
        if summed.write_as_cycles() in '0':
            equiv = True
        
        return equiv
    
    
    def simplify(self):
    
        simplified = []
        prefactors = []
        prefactors_str = []
        
        cycles = self.get('cycle')
        prefacs = self.get('prefactor')
        prefacs_str = self.get('prefactor_str')

        for op_ind in range(len(cycles)):

            op = cycles[op_ind]
            pref = prefacs[op_ind]
            pref_str = prefacs_str[op_ind]
            
            appended = False
            for sim_ind in range(len(simplified)):

                sim = simplified[sim_ind]

                if (op.get('domain') == sim.get('domain')) and (op.get('image') == sim.get('image')):

                    if str(float(prefactors[sim_ind]) + float(pref)) in '0.0000':
                        prefactors[sim_ind] = fr.Fraction(0)
                    else:
                        prefactors[sim_ind] = fr.Fraction(prefactors[sim_ind]) + fr.Fraction(pref)

                    appended = True
                    
                elif (len(op.get('domain')) == 0) and (len(sim.get('domain')) == 0):
                    if str(float(prefactors[sim_ind]) + float(pref)) in '0.0000':
                        prefactors[sim_ind] = fr.Fraction(0)
                    else:
                        prefactors[sim_ind] = fr.Fraction(prefactors[sim_ind]) + fr.Fraction(pref)
                    appended = True

            if not appended:
                    simplified.append(op)
                    prefactors.append(pref)
                    prefactors_str.append(pref_str)

        return CycleSum(simplified, prefactors = prefactors, str_prefactors = prefactors_str)
    
    
    
    
    def act_on(self, operator):
        
        op1 = self.cycle_sum.copy()
        op2 = operator.get_sum().copy()
    
        final_ops = []
        prefactors = []
        prefacs_str = []

        for op1_ind in range(len(op1)):

            prefac1 = op1[op1_ind]['prefactor']
            prefac1_str = op1[op1_ind]['prefactor_str']

            for op2_ind in range(len(op2)):

                prefac2 = op2[op2_ind]['prefactor']
                prefac2_str = op2[op2_ind]['prefactor_str']
                cycle = op1[op1_ind]['cycle'].act_on(op2[op2_ind]['cycle'])

                op = Cycle()
                prefactors.append(fr.Fraction(prefac1*prefac2))
                prefacs_str.append('('+prefac1_str+')' + r"\times" + '('+prefac2_str+')')
                op.set_cycle({'image': cycle.get('image'), 'domain': cycle.get('domain')})

                final_ops.append(op)
                
        cycle_list = CycleSum(final_ops, prefactors = prefactors, str_prefactors = prefacs_str)

        return cycle_list.simplify()
    
    
    
    
    def write_as_deltas(self, left_label = 'x', right_label = 'y'):
    
        deltas = 0
        for op in self.cycle_sum:

            entries_left = op['cycle'].get('image')
            entries_right = op['cycle'].get('domain')
            prefactor = op['prefactor']
            delta = int(prefactor)

            for ind in range(len(entries_left)):

                delta = delta*sp.KroneckerDelta(sp.symbols(left_label + str(entries_left[ind])),
                                                sp.symbols(right_label + str(entries_right[ind])))
            deltas += delta
        return deltas
    
    

    def sub_cycles_as_str(self, op):
        
        left = op['cycle'].get('image')
        right = op['cycle'].get('domain')

        cycles = '('

        if len(right) > 0:
            ind = 0
            ri_start = right[ind]
            ri = ri_start
            count = 0

            cycle = []

            indices = right.copy()

            while count < len(right):

                lf = left[ind]

                indices.remove(lf)
                cycle.append(lf)

                if lf == ri_start:

                    ri_start = 0
                    cycles = cycles + str(cycle).replace('[', '').replace(']',')')

                    cycle = []

                    if len(indices) > 0:
                        ri_start = indices[0]
                        ind = right.index(ri_start)

                        cycles = cycles + '('

                    else: 
                        count = len(right)

                else:
                    ind = right.index(lf)

                count += 1
        if not (cycles[-1] is ')'):
            cycles = cycles + ')'

        return cycles



    def write_as_cycles(self):
        
        ops = self.cycle_sum

        cycles = ''
        for op in ops:
            prefac_str = ''
            prefac = op['prefactor']
            prefac_string = op['prefactor_str']
            if prefac > 0:
                if prefac is 1:
                    prefac_str = '+'
                else:
                    prefac_str = '+' + str(prefac)

            else:
                if prefac is -1:
                    prefac_str = '-'
                else:
                    prefac_str = str(prefac)

            if prefac != 0:
                cycles += prefac_str + op['cycle'].as_str()
                
        if len(cycles) == 0:
            cycles = '0'

        return cycles

In [4]:
import sympy as sp
import gravipy as gp
import numpy as np
import itertools
import tableau_utils as tu
from tableau_utils import *
from cycle import Cycle
import fractions as fr
from cycle_sum import CycleSum

class Symmetriser(CycleSum):

    def __init__(self, num_list, antisym = False, normalise = True):

        perms = list(itertools.permutations(num_list))
        self.domain = num_list

        odd = 0
        if antisym:
            odd = 1

        operator = []
        prefactors = []
        num_list.sort()
        for perm_ind in range(len(perms)):

            op = Cycle()
            op.set_cycle({'image': [x for x in perms[perm_ind]], 'domain': num_list})

            op.remove_single_cycles()
            perm_order = op.permutation_order()

            del_i = int((-1)**(odd*perm_order))
            prefactors.append(del_i)
            operator.append(op)
            

        self.operator = CycleSum(operator, prefactors, normalise = False)

        self.antisym = antisym
        
        if normalise:
            self.normalise()
        
        
        
    def get_total_domain(self):
        
        return self.operator.get_total_domain()
        
        
    def get(self, prpty):
        
        if prpty is 'domain':
            return self.get_total_domain()
        else: 
            return self.operator.get(prpty)
        
    def get_sum(self):
        
        return self.operator.get_sum()
    
    def get_operator(self):
        
        return self.operator
    
    
    def is_antisym(self):
        
        return self.antisym
    
    def is_normalised(self):
        
        return self.operator.is_normalised()

    def set_normalised(self, val):
        
        self.operator.set_normalised(val)
    
    
    def set_operator(self, op):
        
        self.operator = op
        
        
        
    def set_symmetriser_properties(self, op, antisym = False):
        
        self.operator = op
        self.antisym = antisym
        
        
    def reverse(self):
        
        self.operator = self.operator.reverse()
        return self
        

    def normalise(self):
    
        if not self.is_normalised():
            
            n = len(self.domain)
            self.operator = self.operator.multiply_by_constant(fr.Fraction(1,np.prod(range(1, n+1))))
        
            self.set_normalised(True)
    

    
    def act_on(self, op):
        
        op = op.get_operator() # cyc sum is always saved under 'operator'
        
        return self.operator.act_on(op)
    
    
    def is_subsym_of(self, op):
        
        me_antisym = self.antisym
        op_antisym = op.is_antisym()
        
        my_cycles = [self.operator[ind]['cycle'] for ind in range(len(self.operator))]
        op_cycles = []
        if 'Sym' in str(type(op)):
            op_cycles = op.get_operator().get('cycle')
        else:
            op_cycles = op.get('cycle')
        
        subsym = []
        is_subsym = False
        
        if me_antisym is op_antisym:
            
            for my_cyc in my_cycles:
                
                cyc_sub = False
                for op_cyc in op_cycles:
                    
                    if op_cyc.equivalent(my_cyc):
                        cyc_sub = True
                        
                if cyc_sub:
                    subsym.append(1)
                    
        if len(subsym) == len(my_cycles):
            is_subsym = True
            
        return is_subsym
        
    
    
    def write_as_deltas(self, left_label = 'x', right_label = 'y'):
    
        return self.operator.write_as_deltas(left_label = 'x', right_label = 'y')



    def write_as_cycles(self):
        
        return self.operator.write_as_cycles()


In [5]:
import tableau_utils as tu
import numpy as np
import copy as cp
import fractions as fr
#from operator import Operator

class Diagram():
    
    def __init__(self, size_array, multiplicity = 1):
    
        if 0 in size_array:
            size_array.remove(0)
        diag = [[[]]*ind for ind in size_array]

        self.diagram = {'diagram': diag, 'prefactor': fr.Fraction(multiplicity)}

    
    def complex_conjugate(self, N, multiplicity = 1):
        
        max_col = max(self.size_array())
        sizes = []
        for ind in range(N):

            row_len = 0
            if ind < len(self.diagram['diagram']):
                row_len = len(self.diagram['diagram'][-ind])
            sizes.append(max_col - row_len)
        sizes.reverse()
        cc = Diagram(sizes, multiplicity = multiplicity)

        return cc


    def numerator(self, N):

        num = 1

        for ind_r in range(len(self.diagram['diagram'])):

            for ind_c in range(len(self.diagram['diagram'][ind_r])):

                num = num*(N-ind_r+ind_c)

        return num



    def hook_length(self):

        length = 1

        for ind_r in range(len(self.diagram['diagram'])):

            for ind_c in range(len(self.diagram['diagram'][ind_r])):

                row_entries = self.diagram['diagram'][ind_r][ind_c:]
                column_entries = [[] for ind in self.diagram['diagram'][ind_r:] if len(ind) > ind_c]

                num = len(row_entries)+len(column_entries)-1

                length = length*num

        return length


    def dimension(self, N):

        return self.numerator(N)/self.hook_length()


    def simplify_diagram_list(diags):

        simplified = []

        for diag in diags:

            appended = False
            for sim_ind in range(len(simplified)):

                sim = simplified[sim_ind]
                
                if (diag.size_array() == sim.size_array()):

                    simplified[sim_ind].set_property('prefactor', fr.Fraction(sim.get('prefactor')) + fr.Fraction(diag.get('prefactor')))
                    appended = True

            if not appended:
                    simplified.append(diag)

        return simplified


    def direct_sum(diags):

        diagrams = Diagram.simplify_diagram_list(diags)

        return diagrams


    def size_array(self):

        return [len(row) for row in self.diagram['diagram']]
    


    def is_diagram(self):

        diag_sizes = self.size_array()
        sz = list(sorted(diag_sizes.copy(),reverse=1))

        return max([abs(diag_sizes[ind] - sz[ind]) for ind in range(len(sz))]) == 0


    def get_column(self, ind):

        return [row[ind] for row in self.diagram['diagram'] if ind < len(row)]

    def get_row(self, ind):

        return self.diagram['diagram'][ind]
    
    
    def get(self, prpty):
        
        return self.diagram.get(prpty, None)
    
    
    def set_property(self, prpty, val):
        
        if type(val) is type(self.diagram[prpty]):
        
            self.diagram[prpty] = val
        else:
            typ_prop = str(type(self.diagram[prpty]))
            
            print('Error: value type '+str(type(val)) + ' is not '+ prpty + 'type ' + typ_prop)


    def column_admissable(self):

        diag = self.diagram['diagram']
        admissable = True
        num_colmn = np.max([len(row) for row in diag])

        for col_ind in range(num_colmn):

            column = self.get_column(col_ind)
            entries = [[en[0]] for en in column if len(en)>0]

            for ent in entries:

                if len(ent) > 0:

                    entry = ent[0]
                    eq_entries = []
                    for en in entries:
                        if len(en) > 0:
                            if en[0] == entry:
                                eq_entries.append(entry)

                    if len(eq_entries) > 1:
                        return False

        return admissable


    def row_admissable(self):

        diag = self.diagram['diagram']
        admissable = True

        for row_ind in range(len(diag)):

            row = self.get_row(row_ind)

            for ind in range(len(row)):

                ind_col = len(row)-ind-1
                entry = row[ind_col]

                if len(entry) > 0:
                    read_from_entry = []
                    for r_idx in range(len(diag)):

                        if r_idx < row_ind:

                            prev_row = cp.deepcopy([ent[0] for ent in diag[r_idx] if len(ent)>0])
                            if len(prev_row) > 0:
                                prev_row.reverse()

                                for ent in prev_row:
                                    read_from_entry.append(ent)
                        elif r_idx == row_ind:
                            for c_idx in range(len(row)):

                                col_index = len(row) - c_idx - 1

                                if col_index >= ind_col:

                                    ent_this_row = row[col_index]
                                    if len(ent_this_row) > 0:

                                        read_from_entry.append(ent_this_row[0])

                    if len(read_from_entry) > 0:

                        for num_idx in range(len(read_from_entry)):

                            series = []

                            series = read_from_entry[0:num_idx+1]

                            for num in series:

                                nums = [x for x in series if x == num]

                                for les_num_ind in range(num):

                                    les_num = les_num_ind + 1
                                    less_nums = [y for y in series if y == les_num]

                                    if les_num < num:
                                        if len(nums) > len(less_nums):

                                            return False



        return admissable



    def remove_unadmissable_diags(self,diags):

        admissable = []

        for diag in diags:

            if diag.column_admissable():
                if diag.row_admissable():

                    admissable.append(cp.deepcopy(diag))

        return admissable


    def is_equivalent_to(self,diag2):

        diag1 = self
        di1 = diag1.get('diagram')
        di2 = diag2.get('diagram')

        row_equiv = []
        if len(di1) == len(di2):

            rows1 = diag1.size_array()
            rows2 = diag2.size_array()

            if max(abs(rows1[ind] - rows2[ind]) for ind in range(len(rows1))) == 0:

                for r_ind in range(len(rows1)):
                    equiv = False

                    d1_row = diag1.get_row(r_ind)
                    d1_entries = [ent[0] for ent in d1_row if len(ent) > 0]

                    d2_row = diag2.get_row(r_ind)
                    d2_entries = [tr[0] for tr in d2_row if len(tr) > 0]

                    if len(d1_entries) == len(d2_entries):

                        diffs = [abs(d1_entries[ind] - d2_entries[ind]) for ind in range(len(d1_entries))]
                        diffs.append(0)

                        if max(diffs) == 0:
                            equiv = True

                    row_equiv.append(equiv)
            else:
                row_equiv.append(False)
        else:
            row_equiv.append(False)

        equivalent = True
        if False in row_equiv:
            equivalent =  False

        return equivalent


    def remove_duplicates(self,diags):

        no_duplicates = []

        for diag in diags:
            diag_dupd = False

            for no_dups in no_duplicates:

                if diag.is_equivalent_to(no_dups):
                    diag_dupd = True

            if not diag_dupd:
                no_duplicates.append(diag)

        return no_duplicates





    def remove_indices(self,diags):

        new_diags = []
        for diag in diags:

            size = cp.deepcopy(diag).size_array()
            new_diag = Diagram(size, multiplicity = fr.Fraction(diag.get('prefactor')))

            new_diags.append(new_diag)
        return new_diags

            
    


    def direct_multiple(self, diag2, verbose = False, collect_terms = True):

        diag1 = self
        diag2_sizes = diag2.size_array()
        diag2_enum = [[rw+1]*diag2_sizes[rw] for rw in range(len(diag2_sizes))]

        diag2_enum_cp = diag2_enum.copy()

        diags = []

        new_diagrams = [cp.deepcopy(diag1)]

        for d2_r in diag2_enum:
            for d2_ind in range(len(d2_r)):

                new_diags = []
                entry = d2_r[d2_ind]

                for di_ind in range(len(new_diagrams)):

                    di = cp.deepcopy(new_diagrams[di_ind])
                    di_cp = cp.deepcopy(di)

                    for pl in range(len(di.get('diagram'))):

                        diag = cp.deepcopy(di_cp.get('diagram'))
                        diag[pl].append([entry])

                        candidate = Diagram([1], multiplicity = fr.Fraction(di.get('prefactor')))
                        candidate.set_property('diagram', diag)

                        if candidate.is_diagram():
                            new_diags.append(candidate)

                    diag2 = cp.deepcopy(di.get('diagram'))

                    diag2.append([[entry]]) # append also to a new row at the bottom
                    
                    candidate = Diagram([1], multiplicity = fr.Fraction(di.get('prefactor')))
                    candidate.set_property('diagram', diag2)

                    if candidate.is_diagram():
                        new_diags.append(candidate)

                new_diagrams = self.remove_unadmissable_diags(self.remove_duplicates(new_diags))

        rtn_diags = []
        if not verbose:
            rtn_diags = self.remove_indices(new_diagrams)
        else:
            rtn_diags = new_diagrams
            
        if collect_terms:
            rtn_diags = Diagram.direct_sum(rtn_diags)

        return rtn_diags




    def direct_multiple_list(diags, verbose = False, collect_terms = True):

        diags_cp = cp.deepcopy(diags)
        new_diagrams = []

        if len(diags) > 0:
            new_diagrams = [diags[0]]
            diags_cp = diags_cp[1:]

        while len(diags_cp) > 0:

            new_diags = cp.deepcopy(new_diagrams)
            nds = []

            for nd_ind in range(len(new_diags)):

                nd = cp.deepcopy(new_diags[nd_ind])

                multiples = nd.direct_multiple(cp.deepcopy(diags_cp[0]), verbose = verbose, collect_terms = collect_terms)

                for mult in multiples:
                    nds.append(mult)

            diags_cp.remove(diags_cp[0])
            new_diagrams = cp.deepcopy(nds)

        return new_diagrams
    
    
    def first_occurence(self, N):
        
        sizes = self.size_array()
        max_col = max(sizes)
        arr = []
        for ind in range(N):
            if ind < len(sizes):
                arr_row = [1]*(sizes[ind])
                other = [-1]*(max_col - (sizes[ind]))
                row = arr_row + other
                arr.append(row)
            else:
                arr.append([-1]*max_col)
        first_occ = None
        for col in range(max_col):

            mins = 0
            ons = 0

            for ind in range(col+1):
                column_bef = [row[ind] for row in arr if ind < len(row)]
                mins += len([ent for  ent in column_bef if ent == -1])

            for ind in range(max_col - col):
                ind_a = max_col - ind
                column_aft = [row_a[ind_a] for row_a in arr if ind_a < len(row)]
                ons += len([ent for ent in column_aft if ent == 1])

            if (mins == ons) and mins > 0:
                first_occ = mins

        if max_col ==1 and len(sizes) == N:
            first_occ = 0

        return first_occ
    
    

    def write_diagram(self):

        strin = ''

        di = self.get('diagram')
        weight = self.get('prefactor')

        strin = strin + str(weight)+'\n'+str(di[0])[1:-1].replace(',','')+'\n'

        for row_ind in range(len(di)):
            if row_ind+1 < len(di):
                strin = strin + str(di[row_ind+1])[1:-1].replace(',','')+'\n'
        return strin

    
    def write_diagrams(diags):

        strin = ''
        for diag in diags:

            di = diag.get('diagram')
            weight = diag.get('prefactor')

            strin = strin + diag.write_diagram() #str(weight)+'\n'+str(di[0])[1:-1].replace(',','')+'\n'

           # for row_ind in range(len(di)):
           #     if row_ind+1 < len(di):
           #         strin = strin + str(di[row_ind+1])[1:-1].replace(',','')+'\n'
        return strin

We define a diagram by giving a list of empty boxes, read row-wise, such as:

In [6]:
diag = [3,2]

diagram = Diagram(diag)
print(diagram.write_diagram())

1
[] [] []
[] []



We can multiply diagrams

In [8]:
diag1 = [1]
diag2 = [1]

diagram1 = Diagram(diag1)
print(diagram1.dimension(3)*diagram1.dimension(3))

sums = Diagram.direct_multiple_list([diagram1, diagram1, diagram1, diagram1, diagram1, diagram1], verbose = False)
print(Diagram.write_diagrams(Diagram.simplify_diagram_list(sums)))

dims = [diag.dimension(5) for diag in sums]
print(dims)
print(sum(dims))

9.0
1
[] [] [] [] [] []
5
[] [] [] [] []
[]
9
[] [] [] []
[] []
10
[] [] [] []
[]
[]
5
[] [] []
[] [] []
16
[] [] []
[] []
[]
10
[] [] []
[]
[]
[]
5
[] []
[] []
[] []
9
[] []
[] []
[]
[]
5
[] []
[]
[]
[]
[]
1
[]
[]
[]
[]
[]
[]

[210.0, 420.0, 420.0, 420.0, 280.0, 420.0, 420.0, 280.0, 420.0, 175.0, 280.0, 280.0, 280.0, 70.0, 420.0, 420.0, 280.0, 420.0, 175.0, 280.0, 280.0, 280.0, 70.0, 420.0, 175.0, 280.0, 280.0, 50.0, 45.0, 280.0, 280.0, 70.0, 280.0, 50.0, 45.0, 70.0, 45.0, 5.0, 420.0, 420.0, 280.0, 420.0, 175.0, 280.0, 280.0, 280.0, 70.0, 420.0, 175.0, 280.0, 280.0, 50.0, 45.0, 280.0, 280.0, 70.0, 280.0, 50.0, 45.0, 70.0, 45.0, 5.0, 280.0, 280.0, 70.0, 280.0, 50.0, 45.0, 70.0, 45.0, 5.0, 70.0, 45.0, 5.0, 5.0, 0.0]
15625.0


In [9]:
d2_1 = [2,1]
diag2_1 = Diagram(d2_1)

sums_diag2_1 = Diagram.direct_multiple_list([diag2_1, diag2_1], verbose = False)
print(Diagram.write_diagrams(Diagram.simplify_diagram_list(sums_diag2_1)))

1
[] [] [] []
[] []
1
[] [] [] []
[]
[]
1
[] [] []
[] [] []
2
[] [] []
[] []
[]
1
[] [] []
[]
[]
[]
1
[] []
[] []
[] []
1
[] []
[] []
[]
[]



we can complex-conjugate them (for this we need to SU(N) dimension)

In [4]:
cc = diagram1.complex_conjugate(5) # over N = 3
print(cc.write_diagram())

1
[]
[]
[]
[]



and calculate its dimansion and hook length:

In [5]:
dim = cc.dimension(5)
print('dimension: ', dim)
hook = cc.hook_length()
print('hook length: ', hook)

dimension:  5.0
hook length:  24


and calculate the first occurence of any multiplet over some $N$:

In [6]:
print('cc first occurence: ', cc.first_occurence(5))
print('diagram first occurence: ', diagram.first_occurence(5))

cc first occurence:  None
diagram first occurence:  3


## Tableau

Tableau are an extension of diagrams, and we can preform the same operations on them, only they will only use the shape of the diagram so far.

In [28]:
from cycle import Cycle
#from cycle_sum import CycleSum

import numpy as np
import copy as cp
import fractions as fr
#from operator import Operator
from cycle_sum import CycleSum
from symmetriser import Symmetriser


class Tableau(Diagram):
    
    
    def __init__(self, diagram, prefactor = 1, prefactor_str = ''):
    
        pruned_diagram = []
        for row in diagram:
            if len(row) > 0:
                pruned_diagram.append(row)
                
    
        sizes = [len(row) for row in pruned_diagram]
        self.org_diag = pruned_diagram
        diag = Diagram(sizes)
        
        full_diagram = []
        for row in pruned_diagram:
            full_row = []
            for ent in pruned_diagram:
                full_row.append([ent])
            full_diagram.append(full_row)

        diag.set_property('diagram', pruned_diagram)
    
        self.diagram = diag
        self.prefactor = fr.Fraction(prefactor)
        self.prefactor_str = prefactor_str
        if len(prefactor_str) == 0:
            self.prefactor_str = str(prefactor)
            
            
    def get(self, prpty):
        
        if prpty is 'diagram':
            return self.diagram
        elif prpty is 'original_diagram':
            return self. org_diag
        elif prpty is 'prefactor':
            return self.prefactor
        elif prpty is 'prefactor_str':
            return self.prefactor_str
        else:
            return None
        
    def get_row(self, ind):
        return self.diagram.get_row(ind)
    
    def get_column(self, ind):
        return self.diagram.get_column(ind)
        
        
    def set_diagram(self, diag):
        self.diagram = diag
    
    def set_prefactor(self, pref):
        self.prefactor = pref
        
    def set_prefactor_str(self, pref):
        self.prefctor_str = str(pref)
        
        
    def complex_conjugate(self):
        
        cc = self.diagram.complex_conjugate()
        return cc
    
    def dimension(self):
        return self.diagram.dimension()
    
    def hook_length(self):
        return self.diagram.hook_length()
    
    def first_occurence(self):
        return self.diagram.first_occurence()
    
    def size_array(self):
        return self.diagram.size_array()
    
    
    def row_wise_entries(self):
        
        ents = []
        
        diag = self.diagram.get('diagram')
        for row in diag:
            for col in row:
                ents.append(col)
                
        return ents
    
    def column_wise_entries(self):
        
        ents = []
        diag = self.diagram.get('diagram')
        sizes = self.diagram.size_array()
        for col_ind in range(max(sizes)):
            
            column = self.get_column(col_ind)
            for col in column:
                ents.append(col)
                
        return ents
    
    
    def list_of_columns(self):
        
        sizes = self.size_array()
        max_col = max(sizes)
        
        columns = []
        for col_ind in range(max_col):
            columns.append(self.diagram.get_column(col_ind))
            
        return columns
    
    def list_of_rows(self):
        
        sizes = self.size_array()
        
        rows = []
        for row_ind in range(len(sizes)):
            rows.append(self.diagram.get_row(row_ind))
            
        return rows
    
        
    def is_standard(self):
        
        standard = False
        rows = self.list_of_rows()
        columns = self.list_of_columns()
        col_std = True
        row_std = True
        
        for row in rows:
            row_strict = all(i < j for i, j in zip(row, row[1:]))
            
            if not row_strict:
                row_std = False
        
        for column in columns:
            column_strict = all(i < j for i, j in zip(column, column[1:]))
            
            if not column_strict:
                col_std = False
        
        if col_std and row_std:
            standard = True
            
        return standard
        
        
        
    def is_semistandard(self):
        
        standard = False
        rows = self.list_of_rows()
        columns = self.list_of_columns()
        col_std = True
        row_std = True
        
        for row in rows:
            row_strict = all(i <= j for i, j in zip(row, row[1:]))
            
            if not row_strict:
                row_std = False
        
        for column in columns:
            column_strict = all(i < j for i, j in zip(column, column[1:]))
            
            if not column_strict:
                col_std = False
        
        if col_std and row_std:
            standard = True
            
        return standard
    
    
    def parent(self):
        
        diag = cp.deepcopy(self.org_diag)
        print(diag)
        maxval = np.max([np.max(row) for row in diag])
        
        parent_diag = [[val for val in row if val != maxval] for row in diag]
        tab = Tableau(parent_diag, prefactor = self.prefactor, prefactor_str = self.prefactor_str)
        
        return tab
    
    def ancestor(self, n):
        
        tab = self
        if n < len(sum(self.org_diag, [])):
            for ind in range(n):
                tab = tab.parent()
        else:
            print('Error: '+str(n)+"-th ancestor does not exist for tableau of "+ str(len(sum(self.org_diag, []))) + 'boxes.')
        return tab
    
    
    def is_ancestor_of(self, tab):
        
        len_self = len(sum(self.org_diag, []))
        len_tab = len(sum(tab.get('original_diagram'), []))
        
        tab_cp = cp.deepcopy(tab)
        ancestor = False
        while len_self <= len_tab:
            
            if tab_cp.get('original_diagram') == tab.get('original_diagram'):
                ancestor = True
            else:
                tab_cp = tab_cp.parent()
                len_tab = len(sum(tab_cp.get('original_diagram'), []))
        return ancestor
        
            
    def remove_indices(tableau):

        sizes = Diagram.size_array(tableau)

        return Diagram(sizes, multiplicity = tableau.get_property('prefactor'))


    def remove_brackets(ls):

        removed = []

        for rw in ls:

            entries = []

            for ent in rw:
                if len(ent) > 0:
                    entries.append(ent[0])
                else:
                    entries.append(0)

            removed.append(entries)

        return removed
    
    
    def write_diagram(self):

        strin = ''

        di = self.diagram.get('diagram')
        weight = self.diagram.get('prefactor')

        strin = strin + str(weight)+'\n'+str(di[0])[1:-1].replace(',','')+'\n'

        for row_ind in range(len(di)):
            if row_ind+1 < len(di):
                strin = strin + str(di[row_ind+1])[1:-1].replace(',','')+'\n'
        return strin

    
    def write_diagrams(diags):

        strin = ''
        for diag in diags:

            di = diag.get('diagram')
            weight = diag.get('prefactor')

            strin = strin + diag.write_diagram()
        return strin
    
    
    def as_operator(self, order = 'r->l'):
        
        rows = self.list_of_rows()
        columns = self.list_of_columns()
        
        ops = []
        
        for column in columns:
            if len(column) > 1:
                ops.append(Symmetriser(column, antisym = True))
        for row in rows:
            if len(row) > 1:
                ops.append(Symmetriser(row))

        
        prefac = self.prefactor
        prefac_str = self.prefactor_str
        
        return Operator(ops, prefactor = prefac, prefactor_str = prefac_str, order = order)
    
    
    def row_word(self):
        
        word = []
        
        for row in self.org_diag:
            for ent in row:
                word.append(ent)
            
        return word
    
    
    def column_word(self):
        
        word = []
        
        row_len = [len(row) for row in self.org_diag]
        
        for ind in range(np.max(row_len)):
            column = self.get_column(ind)
            
            for ent in column:
                word.append(ent)
                
        return(word)
    
    
    def row_ordered(self):
        
        row_word = self.row_word()
        return all(x<y for x, y in zip(row_word, row_word[1:]))
    
    
    def column_ordered(self):
        
        column_word = self.column_word()
        return all(x<y for x, y in zip(column_word, column_word[1:]))
    
    def row_mold(self):
        
        mold = 0
        tab = cp.deepcopy(self)
        while not tab.row_ordered():
            mold = mold+1
            tab = tab.parent()
            
        return mold
            
            
    def column_mold(self):
    
        mold = 0
        tab = cp.deepcopy(self)
        while not tab.column_ordered():
            mold = mold+1
            tab = tab.parent()
            
        return mold
    
    
    def as_mold_operator(self, normalised = False):
        
        tableau = {'rows': [row for row in self.list_of_rows() if len(row) > 1], 
                   'columns': [col for col in self.list_of_columns() if len(col) > 1]}
        
        row_mold = self.row_mold()
        column_mold = self.column_mold()
        
        mold = []
        
        if row_mold <= column_mold:
            
            if not row_mold%2: # if it's even
                
                mold = [Symmetriser(row) for row in tableau['rows']] +\
                       [Symmetriser(column, antisym = True) for column in tableau['columns']] +\
                       [Symmetriser(row) for row in tableau['rows']]
                
                next_op = {'type': 'columns', 'antisym': True} # antisyms next
                next_op1 = {'type': 'rows', 'antisym': False} # antisyms next
                
                anc = cp.deepcopy(self)

                for ind in range(row_mold):
                    
                    anc = anc.parent()
                    tableau = {'rows': [row for row in anc.list_of_rows() if len(row) > 1], 
                               'columns': [col for col in anc.list_of_columns() if len(col) > 1]}
                    
                    mold = [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]] +\
                           mold +\
                           [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]]
                    
                    next_op2 = next_op # next type of operator
                    next_op = next_op1
                    next_op1 = next_op2
                
            else: # if it's odd
                
                mold = [Symmetriser(column, antisym = True) for column in tableau['columns']] +\
                       [Symmetriser(row) for row in tableau['rows']] +\
                       [Symmetriser(column, antisym = True) for column in tableau['columns']]
                
                next_op = {'type': 'rows', 'antisym': False} # antisyms next
                next_op1 = {'type': 'columns', 'antisym': True} # antisyms next
                
                anc = cp.deepcopy(self)

                for ind in range(row_mold):
                    
                    anc = anc.parent()
                    tableau = {'rows': [row for row in anc.list_of_rows() if len(row) > 1], 
                               'columns': [col for col in anc.list_of_columns() if len(col) > 1]}
                    
                    mold = [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]] +\
                           mold +\
                           [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]]
                    
                    next_op2 = next_op # next type of operator
                    next_op = next_op1
                    next_op1 = next_op2
        
        
        else:
            
            if not column_mold%2: # if it's even
                
                mold = [Symmetriser(column, antisym = True) for column in tableau['columns']] +\
                       [Symmetriser(row) for row in tableau['rows']] +\
                       [Symmetriser(column, antisym = True) for column in tableau['columns']]
                
                next_op = {'type': 'rows', 'antisym': False} # antisyms next
                next_op1 = {'type': 'columns', 'antisym': True} # antisyms next
                
                anc = cp.deepcopy(self)

                for ind in range(row_mold):
                    
                    anc = anc.parent()
                    tableau = {'rows': [row for row in anc.list_of_rows() if len(row) > 1], 
                               'columns': [col for col in anc.list_of_columns() if len(col) > 1]}
                    
                    mold = [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]] +\
                           mold +\
                           [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]]
                    
                    next_op2 = next_op # next type of operator
                    next_op = next_op1
                    next_op1 = next_op2
                
            else: # if it's odd
                
                mold = [Symmetriser(row) for row in tableau['rows']] +\
                       [Symmetriser(column, antisym = True) for column in tableau['columns']] +\
                       [Symmetriser(row) for row in tableau['rows']]
                
                next_op = {'type': 'columns', 'antisym': True} # antisyms next
                next_op1 = {'type': 'rows', 'antisym': False} # antisyms next
                
                anc = cp.deepcopy(self)

                for ind in range(column_mold):
                    
                    anc = anc.parent()
                    tableau = {'rows': [row for row in anc.list_of_rows() if len(row) > 1], 
                               'columns': [col for col in anc.list_of_columns() if len(col) > 1]}
                    
                    mold = [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]] +\
                           mold +\
                           [Symmetriser(dom, antisym = next_op['antisym']) for dom in tableau[next_op['type']]]
                    
                    next_op2 = next_op # next type of operator
                    next_op = next_op1
                    next_op1 = next_op2
        
        prefac = self.prefactor
        prefac_str = self.prefactor_str
        
        mold_op = Operator(mold, prefactor = prefac, prefactor_str = prefac_str, normalised = normalised)
            
        return mold_op
            
        
        
        
        

In [29]:
tabs = [[[1,2,4], [3,5], [6]],
        [[1,2,4], [3,6], [5]],
        [[1,2,5], [3,4], [6]],
        [[1,2,5], [3,6], [4]],
        [[1,2,6], [3,4], [5]],
        [[1,2,6], [3,5], [4]]]

s3 = Operator([Symmetriser([4,5], antisym = True), Symmetriser([1,2]), 
               Symmetriser([4,6]), Symmetriser([1,3], antisym = True),
               Symmetriser([4,5], antisym = True), Symmetriser([1,2])])

molds1 = []
tabls = []

for tab in tabs:
    print('====================================')        
    tableau = Tableau(tab)
    tabls.append(tableau)
    print(tableau.write_diagram())
    
    mold_op = tableau.as_mold_operator()
    molds1.append(mold_op)
    print([[op.domain, op.is_antisym()] for op in mold_op.operator_seq])
    
    print('x')
    for tab2 in tabs:
        if (not (tab2 is tab)) and (not (tab2 in tabls)):
            tableau2 = Tableau(tab2)
            print(tableau2.write_diagram())

            mold_op2 = tableau2.as_mold_operator()
            print([[op.domain, op.is_antisym()] for op in mold_op2.operator_seq])
            print(mold_op.deepact_on(mold_op2.deepact_on(s3)).write_as_cycles())
            print('---------------')

Operator has been normalised.
1
1 2 4
3 5
6

[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[[1, 2], False], [[1, 3], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3], True], [[1, 2], False]]
x
1
1 2 4
3 6
5

[[1, 2, 4], [3, 6], [5]]
[[1, 2, 4], [3], [5]]
[[1, 2, 4], [3]]
[[1, 2, 4], [3, 6], [5]]
[[1, 2, 4], [3], [5]]
[[1, 2, 4], [3]]
[[1, 2], [3]]
[[1, 2, 4], [3, 6], [5]]
[[1, 2, 4], [3], [5]]
[[1, 2, 4], [3]]
[[[1, 2], False], [[1, 3], True], [[1, 2, 4], False], [[1, 3, 5], True], [[2, 6], True], [[1, 2, 4], False], [[3, 6], False], [[1, 3, 5], True], [[2, 6], True], [[1, 2, 4], False], [[1, 3], True], [[1, 2], False]]


KeyboardInterrupt: 

In [58]:
tabs = [[[1,2,4], [3,5], [6]],
        [[1,2,4], [3,6], [5]],
        [[1,2,5], [3,4], [6]],
        [[1,2,5], [3,6], [4]],
        [[1,2,6], [3,4], [5]],
        [[1,2,6], [3,5], [4]]]

s2 = Operator([Symmetriser([4,5]), Symmetriser([1,2]), 
               Symmetriser([4,6], antisym = True), Symmetriser([1,3], antisym = True),
               Symmetriser([4,5]), Symmetriser([1,2])])

molds = []

for tab in tabs:
    print('====================================')        
    tableau = Tableau(tab)
    print(tableau.write_diagram())
    
    mold_op = tableau.as_mold_operator()
    molds.append(mold_op)
    print([[op.domain, op.is_antisym()] for op in mold_op.operator_seq])
    
    print('x')
    for tab2 in tabs:
        if not (tab2 is tab):
            tableau2 = Tableau(tab2)
            print(tableau2.write_diagram())

            mold_op2 = tableau2.as_mold_operator()
            print([[op.domain, op.is_antisym()] for op in mold_op2.operator_seq])
            print(mold_op.deepact_on(mold_op2.deepact_on(s2)).write_as_cycles())
            print('---------------')

Operator has been normalised.
1
1 2 4
3 5
6

[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[[1, 2], False], [[1, 3], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3], True], [[1, 2], False]]
x
1
1 2 4
3 6
5

[[1, 2, 4], [3, 6], [5]]
[[1, 2, 4], [3], [5]]
[[1, 2, 4], [3]]
[[1, 2, 4], [3, 6], [5]]
[[1, 2, 4], [3], [5]]
[[1, 2, 4], [3]]
[[1, 2], [3]]
[[1, 2, 4], [3, 6], [5]]
[[1, 2, 4], [3], [5]]
[[1, 2, 4], [3]]
[[[1, 2], False], [[1, 3], True], [[1, 2, 4], False], [[1, 3, 5], True], [[2, 6], True], [[1, 2, 4], False], [[3, 6], False], [[1, 3, 5], True], [[2, 6], True], [[1, 2, 4], False], [[1, 3], True], [[1, 2], False]]
-1640250(4 6)-1640250(1 2)(4 6)+3280500(4 6 5)+3280500(1 2)(4 6 5)+820

+73483200(4 5)+73483200(1 2)(4 5)-36741600(1 3)(4 5)-36741600(1 3 2)(4 5)+9185400(4 6)+9185400(1 2)(4 6)-36741600(4 6 5)-36741600(1 2)(4 6 5)-4592700(1 3)(4 6)-4592700(1 3 2)(4 6)+18370800(1 3)(4 6 5)+18370800(1 3 2)(4 6 5)-36741600(1 2 3)(4 5)-36741600(2 3)(4 5)-4592700(1 2 3)(4 6)-4592700(2 3)(4 6)+18370800(1 2 3)(4 6 5)+18370800(2 3)(4 6 5)-18370800(4 5 6)-18370800(1 2)(4 5 6)+9185400(1 3)(4 5 6)+9185400(1 3 2)(4 5 6)+9185400(1 2 3)(4 5 6)+9185400(2 3)(4 5 6)+73483200(3 5 4)+73483200(1 2)(3 5 4)-36741600(1 3 5 4)-36741600(1 3 5 4 2)+9185400(3 6 4)+9185400(1 2)(3 6 4)-36741600(3 6 5 4)-36741600(1 2)(3 6 5 4)-4592700(1 3 6 4)-4592700(1 3 6 4 2)+18370800(1 3 6 5 4)+18370800(1 3 6 5 4 2)-36741600(1 2 3 5 4)-36741600(2 3 5 4)-4592700(1 2 3 6 4)-4592700(2 3 6 4)+18370800(1 2 3 6 5 4)+18370800(2 3 6 5 4)-18370800(3 5 6 4)-18370800(1 2)(3 5 6 4)+9185400(1 3 5 6 4)+9185400(1 3 5 6 4 2)+9185400(1 2 3 5 6 4)+9185400(2 3 5 6 4)-36741600(2 5 4)-36741600(1 2 5 4)+73483200(1 3)(2 5 4)+73483200(1 3

-911250(3 6)-911250(1 2)(3 6)+4556250(3 6)(4 5)+4556250(1 2)(3 6)(4 5)+455625(1 3 6)+455625(1 3 6 2)-2278125(1 3 6)(4 5)-2278125(1 3 6 2)(4 5)+911250(3 4 6)+911250(1 2)(3 4 6)-4556250(3 5 4 6)-4556250(1 2)(3 5 4 6)-455625(1 3 4 6)-455625(1 3 4 6 2)+2278125(1 3 5 4 6)+2278125(1 3 5 4 6 2)+455625(1 2 3 6)+455625(2 3 6)-2278125(1 2 3 6)(4 5)-2278125(2 3 6)(4 5)-455625(1 2 3 4 6)-455625(2 3 4 6)+2278125(1 2 3 5 4 6)+2278125(2 3 5 4 6)-2278125(1 4 3 6)-2278125(1 4 3 6 2)-911250(1 5 4 3 6)-911250(1 5 4 3 6 2)+455625(1 4)(3 6)+455625(1 4 2)(3 6)-2278125(1 5 4)(3 6)-2278125(1 5 4 2)(3 6)+911250(1 6)(3 4)+911250(1 6 2)(3 4)-455625(1 6)(3 5 4)-455625(1 6 2)(3 5 4)-455625(1 6 3 4)-455625(1 6 3 4 2)+2278125(1 6 3 5 4)+2278125(1 6 3 5 4 2)+1822500(1 4)(2 3 6)+1822500(1 4 2 3 6)+3189375(1 5 4)(2 3 6)+3189375(1 5 4 2 3 6)-455625(1 6 2 3 4)-455625(1 6)(2 3 4)-1822500(1 6 2 3 5 4)-1822500(1 6)(2 3 5 4)+2278125(2 5)(4 6)+2278125(1 2 5)(4 6)-4556250(2 4 6 5)-4556250(1 2 4 6 5)-1822500(1 3)(2 5)(4 6)-1822

-1093500(4 5 6)-1093500(1 2)(4 5 6)+546750(1 3)(4 5 6)+546750(1 3 2)(4 5 6)+546750(1 2 3)(4 5 6)+546750(2 3)(4 5 6)-1093500(3 5 6 4)-1093500(1 2)(3 5 6 4)+546750(1 3 5 6 4)+546750(1 3 5 6 4 2)+546750(1 2 3 5 6 4)+546750(2 3 5 6 4)+546750(2 5 6 4)+546750(1 2 5 6 4)-1093500(1 3)(2 5 6 4)-1093500(1 3 2 5 6 4)+546750(1 2 5 6 4 3)+546750(2 5 6 4 3)+546750(3 5)(4 6)+546750(1 2)(3 5)(4 6)-273375(1 3 5)(4 6)-273375(1 3 5 2)(4 6)-273375(1 2 3 5)(4 6)-273375(2 3 5)(4 6)+546750(3 6 4 5)+546750(1 2)(3 6 4 5)-273375(1 3 6 4 5)-273375(1 3 6 4 5 2)-273375(1 2 3 6 4 5)-273375(2 3 6 4 5)-273375(2 6 4)(3 5)-273375(1 2 6 4)(3 5)+546750(1 2 6 4 3 5)+546750(2 6 4 3 5)-273375(2 5 3 6 4)-273375(1 2 5 3 6 4)+546750(1 3 6 4 2 5)+546750(1 3 6 4)(2 5)+546750(1 5 6 4 3)+546750(1 5 6 4 3 2)+546750(1 5 6 4)+546750(1 5 6 4 2)-1093500(1 5 6 4)(2 3)-1093500(1 5 6 4 2 3)-273375(1 5 3)(4 6)-273375(1 5 3 2)(4 6)+546750(1 5)(2 3)(4 6)+546750(1 5 2 3)(4 6)-273375(1 6 4 5 3)-273375(1 6 4 5 3 2)+546750(1 6 4 5)+546750(1 6 4 

-24603750(4 6)-24603750(1 2)(4 6)+12301875(1 3)(4 6)+12301875(1 3 2)(4 6)+12301875(1 2 3)(4 6)+12301875(2 3)(4 6)+12301875(2 5)(4 6)+12301875(1 2 5)(4 6)-24603750(1 3)(2 5)(4 6)-24603750(1 3 2 5)(4 6)+12301875(1 2 5 3)(4 6)+12301875(2 5 3)(4 6)-12301875(1 2 6 4 5 3)-12301875(2 6 4 5 3)+24603750(3 6 4)+24603750(1 2)(3 6 4)-12301875(1 3 6 4)-12301875(1 3 6 4 2)-12301875(1 2 3 6 4)-12301875(2 3 6 4)-12301875(2 5)(3 6 4)-12301875(1 2 5)(3 6 4)+24603750(1 3 6 4)(2 5)+24603750(1 3 6 4 2 5)-12301875(1 2 5 3 6 4)-12301875(2 5 3 6 4)+12301875(1 2 6 4)(3 5)+12301875(2 6 4)(3 5)-12301875(1 6 4 3)-12301875(1 6 4 3 2)+12301875(1 6 4)+12301875(1 6 4 2)+12301875(1 6 4 3)(2 5)+12301875(1 6 4 3 2 5)-12301875(1 6 4)(2 5)-12301875(1 6 4 2 5)+12301875(1 5 2 6 4 3)+12301875(1 5)(2 6 4 3)-12301875(1 5 2 6 4)-12301875(1 5)(2 6 4)-24603750(3 5)(4 6)-24603750(1 2)(3 5)(4 6)+12301875(1 3 5)(4 6)+12301875(1 3 5 2)(4 6)+12301875(1 2 3 5)(4 6)+12301875(2 3 5)(4 6)+24603750(3 6 4 5)+24603750(1 2)(3 6 4 5)-12301875(

...

-243000(4 5 6)-243000(1 2)(4 5 6)+121500(1 3)(4 5 6)+121500(1 3 2)(4 5 6)+121500(1 2 3)(4 5 6)+121500(2 3)(4 5 6)+243000(3 5 6 4)+243000(1 2)(3 5 6 4)-121500(1 3 5 6 4)-121500(1 3 5 6 4 2)-121500(1 2 3 5 6 4)-121500(2 3 5 6 4)-121500(1 5 6 4 3)-121500(1 5 6 4 3 2)+121500(1 5 6 4)+121500(1 5 6 4 2)+60750(3 6)+60750(1 2)(3 6)-60750(3 6)(4 5)-60750(1 2)(3 6)(4 5)-30375(1 3 6)-30375(1 3 6 2)+30375(1 3 6)(4 5)+30375(1 3 6 2)(4 5)-60750(3 4 6)-60750(1 2)(3 4 6)+60750(3 5 4 6)+60750(1 2)(3 5 4 6)+30375(1 3 4 6)+30375(1 3 4 6 2)-30375(1 3 5 4 6)-30375(1 3 5 4 6 2)-30375(1 2 3 6)-30375(2 3 6)+30375(1 2 3 6)(4 5)+30375(2 3 6)(4 5)+30375(1 2 3 4 6)+30375(2 3 4 6)-30375(1 2 3 5 4 6)-30375(2 3 5 4 6)-30375(1 4 3 6)-30375(1 4 3 6 2)-30375(1 4)(3 6)-30375(1 4 2)(3 6)+30375(1 5 4)(3 6)+30375(1 5 4 2)(3 6)-151875(1 6)(3 5 4)-151875(1 6 2)(3 5 4)+30375(1 6 3 4)+30375(1 6 3 4 2)-30375(1 6 3 5 4)-30375(1 6 3 5 4 2)+60750(1 4)(2 3 6)+60750(1 4 2 3 6)-30375(1 5 4)(2 3 6)-30375(1 5 4 2 3 6)-30375(1 6 2 3 4)-

-24603750(4 6)-24603750(1 2)(4 6)+12301875(1 3)(4 6)+12301875(1 3 2)(4 6)+12301875(1 2 3)(4 6)+12301875(2 3)(4 6)-24603750(3 5)(4 6)-24603750(1 2)(3 5)(4 6)+12301875(1 3 5)(4 6)+12301875(1 3 5 2)(4 6)+12301875(1 2 3 5)(4 6)+12301875(2 3 5)(4 6)-8201250(3 6 4 5)-8201250(1 2)(3 6 4 5)+4100625(1 3 6 4 5)+4100625(1 3 6 4 5 2)+4100625(1 2 3 6 4 5)+4100625(2 3 6 4 5)-20503125(2 6 4)-20503125(1 2 6 4)+16402500(1 3)(2 6 4)+16402500(1 3 2 6 4)+4100625(1 2 6 4 3)+4100625(2 6 4 3)-20503125(2 6 4)(3 5)-20503125(1 2 6 4)(3 5)+4100625(1 3 5)(2 6 4)+4100625(1 3 5 2 6 4)+16402500(1 2 6 4 3 5)+16402500(2 6 4 3 5)+4100625(2 5 3 6 4)+4100625(1 2 5 3 6 4)-8201250(1 3 6 4 2 5)-8201250(1 3 6 4)(2 5)+4100625(1 2 5)(3 6 4)+4100625(2 5)(3 6 4)-20503125(1 6 4 2)-20503125(1 6 4)+16402500(1 6 4 2 3)+16402500(1 6 4)(2 3)+4100625(1 6 4 3)+4100625(1 6 4 3 2)-20503125(1 6 4 2)(3 5)-20503125(1 6 4)(3 5)+4100625(1 6 4 2 3 5)+4100625(1 6 4)(2 3 5)+16402500(1 6 4 3 5)+16402500(1 6 4 3 5 2)+4100625(1 5 3 6 4 2)+4100625(1 

-6561000(4 6)-6561000(1 2)(4 6)+3280500(1 3)(4 6)+3280500(1 3 2)(4 6)+3280500(1 2 3)(4 6)+3280500(2 3)(4 6)-13122000(4 5 6)-13122000(1 2)(4 5 6)+6561000(1 3)(4 5 6)+6561000(1 3 2)(4 5 6)+6561000(1 2 3)(4 5 6)+6561000(2 3)(4 5 6)-6561000(3 6 4)-6561000(1 2)(3 6 4)+3280500(1 3 6 4)+3280500(1 3 6 4 2)+3280500(1 2 3 6 4)+3280500(2 3 6 4)-13122000(3 5 6 4)-13122000(1 2)(3 5 6 4)+6561000(1 3 5 6 4)+6561000(1 3 5 6 4 2)+6561000(1 2 3 5 6 4)+6561000(2 3 5 6 4)+3280500(2 6 4)+3280500(1 2 6 4)-6561000(1 3)(2 6 4)-6561000(1 3 2 6 4)+3280500(1 2 6 4 3)+3280500(2 6 4 3)+6561000(2 5 6 4)+6561000(1 2 5 6 4)-13122000(1 3)(2 5 6 4)-13122000(1 3 2 5 6 4)+6561000(1 2 5 6 4 3)+6561000(2 5 6 4 3)+3280500(1 6 4 3)+3280500(1 6 4 3 2)+3280500(1 6 4)+3280500(1 6 4 2)-6561000(1 6 4)(2 3)-6561000(1 6 4 2 3)+6561000(1 5 6 4 3)+6561000(1 5 6 4 3 2)+6561000(1 5 6 4)+6561000(1 5 6 4 2)-13122000(1 5 6 4)(2 3)-13122000(1 5 6 4 2 3)+3280500(2 5)(4 6)+3280500(1 2 5)(4 6)-6561000(1 3)(2 5)(4 6)-6561000(1 3 2 5)(4 6)+3280

-3645000(4 5 6)-3645000(1 2)(4 5 6)-7290000(5 6)-7290000(1 2)(5 6)+1822500(1 3)(4 5 6)+1822500(1 3 2)(4 5 6)+3645000(1 3)(5 6)+3645000(1 3 2)(5 6)+1822500(1 2 3)(4 5 6)+1822500(2 3)(4 5 6)+3645000(1 2 3)(5 6)+3645000(2 3)(5 6)+3645000(3 5 6 4)+3645000(1 2)(3 5 6 4)+7290000(3 4)(5 6)+7290000(1 2)(3 4)(5 6)-1822500(1 3 5 6 4)-1822500(1 3 5 6 4 2)-3645000(1 3 4)(5 6)-3645000(1 3 4 2)(5 6)-1822500(1 2 3 5 6 4)-1822500(2 3 5 6 4)-3645000(1 2 3 4)(5 6)-3645000(2 3 4)(5 6)-1822500(1 5 6 4 3)-1822500(1 5 6 4 3 2)-3645000(1 4 3)(5 6)-3645000(1 4 3 2)(5 6)+1822500(1 5 6 4)+1822500(1 5 6 4 2)+3645000(1 4)(5 6)+3645000(1 4 2)(5 6)-911250(3 6)-911250(1 2)(3 6)+911250(3 6)(4 5)+911250(1 2)(3 6)(4 5)+455625(1 3 6)+455625(1 3 6 2)-455625(1 3 6)(4 5)-455625(1 3 6 2)(4 5)+911250(3 4 6)+911250(1 2)(3 4 6)-911250(3 5 4 6)-911250(1 2)(3 5 4 6)-455625(1 3 4 6)-455625(1 3 4 6 2)+455625(1 3 5 4 6)+455625(1 3 5 4 6 2)+455625(1 2 3 6)+455625(2 3 6)-455625(1 2 3 6)(4 5)-455625(2 3 6)(4 5)-455625(1 2 3 4 6)-45562

+874800(4 5)+874800(1 2)(4 5)-437400(1 3)(4 5)-437400(1 3 2)(4 5)-437400(1 2 3)(4 5)-437400(2 3)(4 5)+218700(4 5 6)+218700(1 2)(4 5 6)-109350(1 3)(4 5 6)-109350(1 3 2)(4 5 6)-109350(1 2 3)(4 5 6)-109350(2 3)(4 5 6)+874800(3 5 4)+874800(1 2)(3 5 4)-437400(1 3 5 4)-437400(1 3 5 4 2)-437400(1 2 3 5 4)-437400(2 3 5 4)+218700(3 5 6 4)+218700(1 2)(3 5 6 4)-109350(1 3 5 6 4)-109350(1 3 5 6 4 2)-109350(1 2 3 5 6 4)-109350(2 3 5 6 4)-437400(2 5 4)-437400(1 2 5 4)+874800(1 3)(2 5 4)+874800(1 3 2 5 4)-437400(1 2 5 4 3)-437400(2 5 4 3)-109350(2 5 6 4)-109350(1 2 5 6 4)+218700(1 3)(2 5 6 4)+218700(1 3 2 5 6 4)-109350(1 2 5 6 4 3)-109350(2 5 6 4 3)-874800(3 5)-874800(1 2)(3 5)-874800(3 4 5)-874800(1 2)(3 4 5)+437400(1 3 5)+437400(1 3 5 2)+437400(1 3 4 5)+437400(1 3 4 5 2)-328050(3 5)(4 6)-328050(1 2)(3 5)(4 6)-437400(3 4 6 5)-437400(1 2)(3 4 6 5)+164025(1 3 5)(4 6)+164025(1 3 5 2)(4 6)+218700(1 3 4 6 5)+218700(1 3 4 6 5 2)+437400(1 2 3 5)+437400(2 3 5)+437400(1 2 3 4 5)+437400(2 3 4 5)+164025(1 2 3 

In [51]:
tabs = [[[1,2,3], [4,5], [6]],
        [[1,2,3], [4, 6], [5]],
        [[1,2,4], [3,5], [6]],
        [[1,2,4], [3,6], [5]],
        [[1,2,5], [3,4], [6]],
        [[1,2,5], [3,6], [4]],
        [[1,2,6], [3,4], [5]],
        [[1,2,6], [3,5], [4]],
        [[1,3,4], [2,5], [6]],
        [[1,3,4], [2,6], [5]],
        [[1,3,5], [2,4], [6]],
        [[1,3,5], [2,6], [4]],
        [[1,3,6], [2,4], [5]],
        [[1,3,6], [2,5], [4]],
        [[1,4,5], [2,6], [3]],
        [[1,4,6], [2,5], [3]]]

s2 = Operator([Symmetriser([4,5]), Symmetriser([1,2]), 
               Symmetriser([4,6], antisym = True), Symmetriser([1,3], antisym = True),
               Symmetriser([4,5]), Symmetriser([1,2])])

for tab in tabs:
    print('-----')
    tableau = Tableau(tab)
    print(tableau.write_diagram())

    mold_op = tableau.as_mold_operator()
    print([[op.domain, op.is_antisym()] for op in mold_op.operator_seq])
    print(mold_op.deepact_on(s2).write_as_cycles())

Operator has been normalised.
-----
1
1 2 3
4 5
6

[[1, 2, 3], [4, 5], [6]]
[[1, 2, 3], [4, 5]]
[[1, 2, 3], [4]]
[[[1, 2, 3], False], [[4, 5], False], [[1, 4, 6], True], [[2, 5], True], [[1, 2, 3], False], [[4, 5], False]]
0
-----
1
1 2 3
4 6
5

[[1, 2, 3], [4, 6], [5]]
[[1, 2, 3], [4, 6], [5]]
[[1, 2, 3], [4], [5]]
[[1, 2, 3], [4]]
[[1, 2, 3], [4, 6], [5]]
[[[1, 2, 3], False], [[1, 4, 5], True], [[2, 6], True], [[1, 2, 3], False], [[4, 6], False], [[1, 4, 5], True], [[2, 6], True], [[1, 2, 3], False]]
0
-----
1
1 2 4
3 5
6

[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[[1, 2], False], [[1, 3], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3], True], [[1, 2], False]]
+945()+945(1 2)+945(4

[[[1, 2], False], [[1, 3], True], [[1, 2, 4], False], [[1, 3, 5], True], [[2, 6], True], [[1, 2, 4], False], [[3, 6], False], [[1, 3, 5], True], [[2, 6], True], [[1, 2, 4], False], [[1, 3], True], [[1, 2], False]]
+405()+405(1 2)+405(4 5)+405(1 2)(4 5)-405/2(1 3)-405/2(1 2 3)-405/2(1 3)(4 5)-405/2(1 2 3)(4 5)-540(4 6)-540(1 2)(4 6)-540(4 5 6)-540(1 2)(4 5 6)+270(1 3)(4 6)+270(1 2 3)(4 6)+270(1 3)(4 5 6)+270(1 2 3)(4 5 6)-405/2(1 3 2)-405/2(2 3)-405/2(1 3 2)(4 5)-405/2(2 3)(4 5)+270(1 3 2)(4 6)+270(2 3)(4 6)+270(1 3 2)(4 5 6)+270(2 3)(4 5 6)+135(4 6 5)+135(1 2)(4 6 5)+135(5 6)+135(1 2)(5 6)-135/2(1 3)(4 6 5)-135/2(1 2 3)(4 6 5)-135/2(1 3)(5 6)-135/2(1 2 3)(5 6)-135/2(1 3 2)(4 6 5)-135/2(2 3)(4 6 5)-135/2(1 3 2)(5 6)-135/2(2 3)(5 6)+675/2(2 4)+675/2(1 4 2)+675/2(2 4 5)+675/2(1 4 5 2)-270(1 3)(2 4)-270(1 4 2 3)-270(1 3)(2 4 5)-270(1 4 5 2 3)-450(2 4 6)-450(1 4 6 2)-450(2 4 5 6)-450(1 4 5 6 2)+360(1 3)(2 4 6)+360(1 4 6 2 3)+360(1 3)(2 4 5 6)+360(1 4 5 6 2 3)-135/2(1 3 4 2)-135/2(2 3 4)-135

[[[1, 2], False], [[3, 4], False], [[1, 3], True], [[2, 4], True], [[1, 2, 5], False], [[3, 4], False], [[1, 3, 6], True], [[2, 4], True], [[1, 2, 5], False], [[3, 4], False], [[1, 3], True], [[2, 4], True], [[1, 2], False], [[3, 4], False]]
+756()+756(1 2)+756(4 5)+756(1 2)(4 5)-378(1 3)-378(1 2 3)-378(1 3)(4 5)-378(1 2 3)(4 5)-432(4 6)-432(1 2)(4 6)-432(4 5 6)-432(1 2)(4 5 6)+216(1 3)(4 6)+216(1 2 3)(4 6)+216(1 3)(4 5 6)+216(1 2 3)(4 5 6)-378(1 3 2)-378(2 3)-378(1 3 2)(4 5)-378(2 3)(4 5)+216(1 3 2)(4 6)+216(2 3)(4 6)+216(1 3 2)(4 5 6)+216(2 3)(4 5 6)-324(4 6 5)-324(1 2)(4 6 5)-324(5 6)-324(1 2)(5 6)+162(1 3)(4 6 5)+162(1 2 3)(4 6 5)+162(1 3)(5 6)+162(1 2 3)(5 6)+162(1 3 2)(4 6 5)+162(2 3)(4 6 5)+162(1 3 2)(5 6)+162(2 3)(5 6)+756(3 4)+756(1 2)(3 4)+756(3 4 5)+756(1 2)(3 4 5)-378(1 4 3)-378(1 2 4 3)-378(1 4 5 3)-378(1 2 4 5 3)-432(3 4 6)-432(1 2)(3 4 6)-432(3 4 5 6)-432(1 2)(3 4 5 6)+216(1 4 6 3)+216(1 2 4 6 3)+216(1 4 5 6 3)+216(1 2 4 5 6 3)-378(1 4 3 2)-378(2 4 3)-378(1 4 5 3 2)-378(

[[[1, 2], False], [[1, 3, 4], True], [[1, 2, 5], False], [[3, 6], False], [[1, 3, 4], True], [[2, 6], True], [[1, 2, 5], False], [[3, 6], False], [[1, 3, 4], True], [[1, 2], False]]
+225()+225(1 2)+225(4 5)+225(1 2)(4 5)-225/2(1 3)-225/2(1 2 3)-225/2(1 3)(4 5)-225/2(1 2 3)(4 5)-225/2(1 3 2)-225/2(2 3)-225/2(1 3 2)(4 5)-225/2(2 3)(4 5)-225(4 6 5)-225(1 2)(4 6 5)-225(5 6)-225(1 2)(5 6)+225/2(1 3)(4 6 5)+225/2(1 2 3)(4 6 5)+225/2(1 3)(5 6)+225/2(1 2 3)(5 6)+225/2(1 3 2)(4 6 5)+225/2(2 3)(4 6 5)+225/2(1 3 2)(5 6)+225/2(2 3)(5 6)-225(3 4)-225(1 2)(3 4)-225(3 4 5)-225(1 2)(3 4 5)+225/2(1 4 3)+225/2(1 2 4 3)+225/2(1 4 5 3)+225/2(1 2 4 5 3)+225/2(1 4 3 2)+225/2(2 4 3)+225/2(1 4 5 3 2)+225/2(2 4 5 3)+225(3 4 6 5)+225(1 2)(3 4 6 5)+225(3 4)(5 6)+225(1 2)(3 4)(5 6)-225/2(1 4 6 5 3)-225/2(1 2 4 6 5 3)-225/2(1 4 3)(5 6)-225/2(1 2 4 3)(5 6)-225/2(1 4 6 5 3 2)-225/2(2 4 6 5 3)-225/2(1 4 3 2)(5 6)-225/2(2 4 3)(5 6)+225/2(1 3 4)+225/2(1 2 3 4)+225/2(1 3 4 5)+225/2(1 2 3 4 5)-225/2(1 4)-225/2(1 2 4)-225

[[[1, 2], False], [[3, 4], False], [[1, 3, 5], True], [[2, 4], True], [[1, 2, 6], False], [[3, 4], False], [[1, 3, 5], True], [[2, 4], True], [[1, 2], False], [[3, 4], False]]
+9()+9(1 2)+9(4 5)+9(1 2)(4 5)-9/2(1 3)-9/2(1 2 3)-9/2(1 3)(4 5)-9/2(1 2 3)(4 5)-9/2(1 3 2)-9/2(2 3)-9/2(1 3 2)(4 5)-9/2(2 3)(4 5)-9(4 6 5)-9(1 2)(4 6 5)-9(5 6)-9(1 2)(5 6)+9/2(1 3)(4 6 5)+9/2(1 2 3)(4 6 5)+9/2(1 3)(5 6)+9/2(1 2 3)(5 6)+9/2(1 3 2)(4 6 5)+9/2(2 3)(4 6 5)+9/2(1 3 2)(5 6)+9/2(2 3)(5 6)+9(3 4)+9(1 2)(3 4)+9(3 4 5)+9(1 2)(3 4 5)-9/2(1 4 3)-9/2(1 2 4 3)-9/2(1 4 5 3)-9/2(1 2 4 5 3)-9/2(1 4 3 2)-9/2(2 4 3)-9/2(1 4 5 3 2)-9/2(2 4 5 3)-9(3 4 6 5)-9(1 2)(3 4 6 5)-9(3 4)(5 6)-9(1 2)(3 4)(5 6)+9/2(1 4 6 5 3)+9/2(1 2 4 6 5 3)+9/2(1 4 3)(5 6)+9/2(1 2 4 3)(5 6)+9/2(1 4 6 5 3 2)+9/2(2 4 6 5 3)+9/2(1 4 3 2)(5 6)+9/2(2 4 3)(5 6)-9/2(2 4)-9/2(1 4 2)-9/2(2 4 5)-9/2(1 4 5 2)+9(1 3)(2 4)+9(1 4 2 3)+9(1 3)(2 4 5)+9(1 4 5 2 3)-9/2(1 3 4 2)-9/2(2 3 4)-9/2(1 3 4 5 2)-9/2(2 3 4 5)+9/2(2 4 6 5)+9/2(1 4 6 5 2)+9/2(2 4)(5 6)+9

[[[1, 2], False], [[1, 3, 4], True], [[2, 5], True], [[1, 2, 6], False], [[3, 5], False], [[1, 3, 4], True], [[2, 5], True], [[1, 2, 6], False], [[3, 5], False], [[1, 3, 4], True], [[2, 5], True], [[1, 2], False]]
+405()+405(1 2)+405(4 5)+405(1 2)(4 5)-405/2(1 3)-405/2(1 2 3)-405/2(1 3)(4 5)-405/2(1 2 3)(4 5)-270(4 6)-270(1 2)(4 6)-270(4 5 6)-270(1 2)(4 5 6)+135(1 3)(4 6)+135(1 2 3)(4 6)+135(1 3)(4 5 6)+135(1 2 3)(4 5 6)-405/2(1 3 2)-405/2(2 3)-405/2(1 3 2)(4 5)-405/2(2 3)(4 5)+135(1 3 2)(4 6)+135(2 3)(4 6)+135(1 3 2)(4 5 6)+135(2 3)(4 5 6)-135(4 6 5)-135(1 2)(4 6 5)-135(5 6)-135(1 2)(5 6)+135/2(1 3)(4 6 5)+135/2(1 2 3)(4 6 5)+135/2(1 3)(5 6)+135/2(1 2 3)(5 6)+135/2(1 3 2)(4 6 5)+135/2(2 3)(4 6 5)+135/2(1 3 2)(5 6)+135/2(2 3)(5 6)-405/2(2 5)-405/2(1 5 2)-405/2(2 5 4)-405/2(1 5 4 2)+405/2(1 3)(2 5)+405/2(1 5 2 3)+405/2(1 3)(2 5 4)+405/2(1 5 4 2 3)+135(2 5)(4 6)+135(1 5 2)(4 6)+135(2 5 6 4)+135(1 5 6 4 2)-405/2(1 3)(2 5)(4 6)-405/2(1 5 2 3)(4 6)-405/2(1 3)(2 5 6 4)-405/2(1 5 6 4 2 3)+135

[[[1, 2], True], [[1, 3], False], [[1, 2], True], [[1, 3, 4], False], [[2, 5], False], [[1, 2, 6], True], [[3, 5], True], [[1, 3, 4], False], [[2, 5], False], [[1, 2, 6], True], [[3, 5], True], [[1, 3, 4], False], [[2, 5], False], [[1, 2], True], [[1, 3], False], [[1, 2], True]]
0
-----
1
1 3 4
2 6
5

[[1, 3, 4], [2, 6], [5]]
[[1, 3, 4], [2], [5]]
[[1, 3, 4], [2]]
[[1, 3], [2]]
[[1, 3, 4], [2, 6], [5]]
[[1, 3, 4], [2], [5]]
[[1, 3, 4], [2, 6], [5]]
[[1, 3, 4], [2], [5]]
[[1, 3, 4], [2]]
[[1, 3], [2]]
[[[1, 2], True], [[1, 3], False], [[1, 2], True], [[1, 3, 4], False], [[1, 2, 5], True], [[3, 6], True], [[1, 3, 4], False], [[2, 6], False], [[1, 2, 5], True], [[3, 6], True], [[1, 3, 4], False], [[1, 2], True], [[1, 3], False], [[1, 2], True]]
0
-----
1
1 3 5
2 4
6

[[1, 3, 5], [2, 4], [6]]
[[1, 3, 5], [2, 4]]
[[1, 3], [2, 4]]
[[1, 3], [2]]
[[1, 3, 5], [2, 4], [6]]
[[1, 3, 5], [2, 4], [6]]
[[[1, 2], True], [[3, 4], True], [[1, 3, 5], False], [[2, 4], False], [[1, 2, 6], True], [[3, 4], T

In [18]:
not 4%2

True

In [31]:
def type_of_op(op):
    
    if op.is_antisym():
        return 'antisymmetriser'
    else:
        return 'symmetriser'

operators = ['a1','a2','a3', 's1', 's2', 'a4', 'a5', 's3','s4', 's5']
lis_sep = [[operators[0]]]
lis_sep_ind = 0

last_ent = type_ent(operators[0])

for ind in range(len(operators)):
    if ind > 0:
        entry = operators[ind]
        
        if type_ent(entry) is last_ent:
            
            lis_sep[lis_sep_ind].append(entry)
        else:
            lis_sep.append([entry])
            lis_sep_ind = lis_sep_ind + 1
            
        last_ent = type_ent(entry)
print(lis_sep)
            
    

[['a1', 'a2', 'a3'], ['s1', 's2'], ['a4', 'a5'], ['s3', 's4', 's5']]


In [21]:
from cycle import Cycle
from cycle_sum import CycleSum
from symmetriser import Symmetriser
from diagram import Diagram
from tableau import Tableau
import numpy as np
import copy as cp
import fractions as fr
import tableau_utils as tu

class Operator():
    
    def __init__(self, ops, prefactor = 1, prefactor_str = '', order = 'r->l', normalised = True):
        
        self.domain = list(set(sum([op.get_total_domain() for op in ops if len(ops) > 0], [])))
        
        self.operator_seq = ops
        
        self.order = order
        self.prefactor = fr.Fraction(prefactor)
        self.prefactor_str = prefactor_str
        self.normalised = False
        self.idempotent = False
        if normalised:
            self.idempotent = self.is_idempotent()
            if self.idempotent:
                if normalised:
                    self.normalise()
                else:
                    self.normalised = False
        
    def is_idempotent(self):
        
        idem = False
        
        self_cp = cp.deepcopy(self)
        
        collapsed = self_cp.collapse()
        if collapsed.is_idempotent():
            idem = True
            
        return idem
    
    def is_normalised(self):
        return self.normalised
        
    
    def set_normalised(self, val):
        
        self.normalised = val
        
        
    def set_prefactor(self, val):
        
        self.prefactor = val
        

    def get(self, prpty):
        
        if prpty is 'prefactor':
            return self.prefactor
        elif prpty is 'prefactor_str':
            return self.prefactor_str
        elif prpty is 'symmetrisers':
            return Operator([sym for sym in self.operator_seq if not sym.is_antisym()])
        elif prpty is 'antisymmetrisers':
            return Operator([asym for asym in self.operator_seq if sym.is_antisym()])
        elif prpty is 'domain':
            return self.domain
        elif prpty is 'operator':
            return self.operator_seq
        elif prpty is 'order':
            return self.order
        
    
        
    def normalise(self):
        
        norm_fac = None
        normalised = cp.deepcopy(self)
        
        if self.is_idempotent():
            if (not normalised.is_normalised()):
            
                self_cp = cp.deepcopy(self)

                norm_fac = self_cp.collapse().get_normalisation_factor()

                self.set_normalised(True)
                #self.operator_seq = self_cp.multiply_by_constant(fr.Fraction(1))
                self.set_prefactor(norm_fac)
                print('Operator has been normalised.')
        
    

    def deepact_on(self, op):
        
        operator = op
        op_order = 'r->l'
        if 'Operator' in str(type(op)):
            op_order = op.get('order')
            operator = op.collapse()

        if operator is not None:
            
            my_order = self.order
            
            if not my_order is op_order:
                operator = operator.reverse()
                
            my_collapsed = self.collapse()

            deepact = my_collapsed.act_on(operator)
            
        return deepact

            
    def collapse(self):
        
        op_seq = self.operator_seq
        prefactor = self.prefactor
        
        collapsed = cp.deepcopy(op_seq[0]).multiply_by_constant(prefactor)
        order = self.order
        for ind in range(len(op_seq)):
            
            if ind > 0:
                collapsed = collapsed.act_on(op_seq[ind])
                
        return collapsed
            

    
    def as_tableau(self):
        
        syms = self.operator['symmetrisers']
        asyms = self.operator['antisymmetrisers']
        
        prefac = self.operator['prefactor']
        prefac_str = self.operator['prefactor_str']
        
        
        sym_domains = self.sym_multiplets
        sdom_lengths = [len(dom) for dom in sym_domains]
        asym_domains = self.asym_multiplets
        adom_lengths = [len(dom) for dom in asym_domains]
        
        sorted_sdoms = [x for _,x in sorted(zip(sdom_lengths,sym_domains))]
        sorted_adoms = [x for _,x in sorted(zip(adom_lengths,asym_domains))]
        
        tab_sym = sorted_sdoms
        tab_asym = []
        
        for ind in range(len(sorted_adoms[0])):
            tab_asym[ind] = []
            for adom in sorted_adoms:
                if ind < len(adom):
                    tab_asym[ind].append(adom[ind])
                    
        print(tab_sym)
        print(sorted_adoms)
        
        
        
    def draw_symmetriser(self, lines, draw_cfg):
        
        
        return None
    
    
    
    def categorise_operators(self):
        
        op_seq = self.operator_seq
        op_types = [tu.get_type(str(type(op))) for op in op_seq]
        print(op_types)
        
        categories = []
        
        if len(op_seq) > 0:
            category = [op_seq[0]]
            typ = op_types[0]
            for op_ind in range(len(op_seq)):

                if op_ind > 0:
                    op = op_seq[op_ind]
                    op_typ = op_types[op_ind]
                    
                    if typ is op_typ:
                        category.append(op)
                    else:
                        print(category)
                        categories.append(category)
                        category = [op]
                        typ = op_typ
                        
        return categories 
            
            
            
    def draw_invertical_operators(self, draw_cfg, strin, lines):
    
        line_width = draw_cfg['line width']
        vert_sep = draw_cfg['vertical separation']
        horiz_sep = draw_cfg['horizontal separation']
        cycle_len = draw_cfg['cycle length']
        sym_width = draw_cfg['symmetriser width']
        perm_dist = draw_cfg['permutation distance']
        
        
    def draw_cycle(self, draw_cfg, strin, lines, cycle):
    
        return None
        
        
        
    def write_latex1(self, scale = 1, write_prefac = False):
        
        draw_cfg = tu.read_yaml('latex_config.yaml')

        strin = draw_cfg['start string']
        line_width = draw_cfg['line width']
        vert_sep = draw_cfg['vertical separation']
        horiz_sep = draw_cfg['horizontal separation']
        cycle_len = draw_cfg['cycle length']
        sym_width = draw_cfg['symmetriser width']
        perm_dist = draw_cfg['permutation distance']
        
        domain = np.arange(min(self.domain), max(self.domain)+1)
        lines = {str(num): [(num)*vert_sep*scale, 0, (len(domain) - num + 1)*vert_sep*scale] \
                 for num in domain}

        op_ind = 0
        print(len(self.operator_seq))
        while op_ind < len(self.operator_seq):
            print('fir', op_ind)
            op = self.operator_seq[op_ind]
            typ = str(type(op))
            cycle = None
            dom = op.get("domain")
            print('domain', dom)
            perm_str = ''
            extra_draw = ''
            
            next_op = None
            
            if "ymmetriser" in typ:
                asym = asym = op.is_antisym()
                print('----' + str(asym) + '---- at ind: '+str(op_ind))
                next_asym = asym
                op_ind_x = op_ind
                op = self.operator_seq[op_ind_x]
                dom = op.get("domain")
                prev_top = min(dom)
                prev_bottom = min(dom) - 1
                
                exit = False
                cycle = {'domain': domain, 'image': list(np.zeros(len(domain)))}
                
                while (op_ind_x < len(self.operator_seq)) and (not exit):

                    op = self.operator_seq[op_ind_x]
                    
                    if ("ymmetriser" in str(type(self.operator_seq[op_ind_x]))):
                        
                        next_asym = op.is_antisym()
                        dom = op.get("domain")
                        image = dom
                        print('next op type: '+ str(next_asym) + 'at ind: '+ str(op_ind_x))

                        if next_asym == asym:

                            vert = vert_sep*0.4
                            horiz = abs((horiz_sep - sym_width)/2)
                            asym = op.is_antisym()
                            sym_str = 'symmetrizer'

                            start = np.round(lines[list(lines.keys())[-1]][1] - horiz, 3)
                            end = np.round(lines[list(lines.keys())[-1]][1] - horiz - sym_width, 3)
                            
                            top_line = max([min(dom), prev_bottom+1])
                            prev_top = top_line
                            top = np.round(lines[str(top_line)][2] + vert, 4)
                            
                            bottom_line = top_line + len(dom) - 1
                            bottom = np.round(lines[str(bottom_line)][2] - vert,4)
                            prev_bottom = top_line + len(dom) - 1

                            if asym:
                                sym_str = 'anti'+sym_str
                                
                            extra_draw += "\draw["+ sym_str + "](" + str(end) + "," + str(top) + ")rectangle(" +\
                                         str(start) + "," + str(bottom) +");" + "\n"
                            
                                                        
                            inner_lines = list(np.arange(top_line, bottom_line+1))
                            for ind in range(len(cycle['domain'])):
                                
                                entry = cycle['domain'][ind]
                                if (cycle['image'][ind]-0.5) < 0:
                                    if entry in dom:
                                    
                                        for dom_ent_ind in range(len(dom)):
                                        
                                            dom_ent = dom[dom_ent_ind]
                                            if entry == dom_ent:
                                                cycle['image'][ind] = inner_lines[dom_ent_ind]
                            
                        else:
                            exit = True
                            op_ind = op_ind_x
                    
                    else:
                        exit = True
                        op_ind = op_ind_x
                        
                    print('op_ind',op_ind)
                    
                    op_ind_x += 1
                    if (op_ind_x >= len(self.operator_seq)) and (not exit):
                        op_ind = op_ind_x
                        
                    print('new x', op_ind_x)

                missing_lines = [cycle['domain'][ind] for ind in range(len(cycle['domain'])) \
                                 if not cycle['domain'][ind] in cycle['image']]
                print('miss lin', missing_lines)
                for lin in missing_lines:
                    ex = False
                    for ind in range(len(cycle['image'])):
                        
                        
                        if cycle['image'][ind] - 0.5 < 0 and (not ex):
                            cycle['image'][ind] = lin
                            ex = True
                print('final cyc', cycle)
            
            elif "Cycle" in str(type(op)):
                image = op.get("image")
                
                
            print(extra_draw)  
            print('cycle', cycle)    
            doma = domain
            imag = []
            for dm in doma:
                
                if dm in dom:
                    for ind in range(len(dom)):
                        if dm == dom[ind]:
                            imag.append(image[ind])
                else:
                    imag.append(dm)
            
            for ind in range(len(doma)):
                line = lines[str(doma[ind])][0]
                start = np.round(lines[str(doma[ind])][1], 3)
                target_line = lines[str(imag[ind])][0]
                end = np.round(start - horiz_sep*scale, 3)
                
                to_str = ''
                if line != target_line:
                    to_str = "[wavy]"
                    
                perm_str += "\draw[line width =" + str(line_width) + "pt](" + str(end) +\
                            "," + str(target_line) + ")to"+ to_str +"("+ str(start) + "," + str(line) +");" +"\n"
                
                lines[str(doma[ind])][1] = end
                
            perm_str = perm_str + extra_draw
            
            strin += perm_str
            
        return strin + "\end{tikzpicture}"
    
    
    def type_of_op(op):
    
        if op.is_antisym():
            return 'antisymmetriser'
        else:
            return 'symmetriser'
    
    def separate_operators(self):

        operators = self.operator_seq
        lis_sep = [[operators[0]]]
        lis_sep_ind = 0

        last_ent = type_ent(operators[0])

        for ind in range(len(operators)):
            if ind > 0:
                entry = operators[ind]

                if type_ent(entry) is last_ent:

                    lis_sep[lis_sep_ind].append(entry)
                else:
                    lis_sep.append([entry])
                    lis_sep_ind = lis_sep_ind + 1

                last_ent = type_ent(entry)
                
        return lis_sep
    
    def sort_operators(ops, domain):
        
        domains = [op.domain() for op in ops]
        scores = [sum(dom) for dom in domains]
        
        sorted_ops = [x for _, x in sorted(zip(scores, ops))]
        cyc_images = [x for _, x in sorted(zip(scores, domains))]
        
        cyc_img = []
        for dom in cyc_images:
            for ent in dom:
                cyc_img.append(ent)
                
        for ind in domain:
            fuck = ''
        
        return cyc_img, sorted_ops
    
    
    def draw_operators(ops, domain, lines, config):
        
        strin = ''
        
        
    
    def draw_cycle(cyc_dom, cyc_img, domain, lines, config):
    
        line_width = draw_cfg['line width']
        vert_sep = draw_cfg['vertical separation']
        horiz_sep = draw_cfg['horizontal separation']
        cycle_len = draw_cfg['cycle length']
        perm_dist = draw_cfg['permutation distance']
        strin = ''
        for ind in range(len(cyc_dom)):
            
            line_dom = cyc_dom[ind]
            start_x = lines[str(line_dom)][1]
            start_y = lines[str(line_dom)][2]
            
            line_img = cyc_img[ind]
            end_x = lines[str(line_img)][1] + cycle_len
            lines[str(line_img)][1] += cycle_len
            end_y = lines[str(line_img)][2]
            
            wavy = ''
            
            if line_dom != line_img:
                wavy = '[wavy]'
                
            strin = strin + "\draw[line width =" + line_width +"](" + start_x + "," + start_y +")to" +\
                    wavy + "(" + end_x + "," + end_y + ");"
            
        return strin
    
    def write_latex(self, scale = 1, write_prefac = False):
        
        draw_cfg = tu.read_yaml('latex_config.yaml')

        strin = draw_cfg['start string']
        line_width = draw_cfg['line width']
        vert_sep = draw_cfg['vertical separation']
        horiz_sep = draw_cfg['horizontal separation']
        cycle_len = draw_cfg['cycle length']
        sym_width = draw_cfg['symmetriser width']
        perm_dist = draw_cfg['permutation distance']
        
        domain = np.arange(min(self.domain), max(self.domain)+1)
        lines = {str(num): [(num)*vert_sep*scale, 0, (len(domain) - num + 1)*vert_sep*scale] \
                 for num in domain}
        
        operators_list = self.separate_operators()
        
        cyc_dom = domain
        
        for operators in operators_list:
            
            cyc_img, ops_sorted = sort_operators(operators)
            
            cyc_strin = draw_cycle(cyc_dom, cyc_img, domain, lines, draw_cfg)
            op_strin = draw_operators(ops_sorted, domain, lines, draw_cfg)
            
            cyc_dom = cyc_img
            strin = strin + cyc_strin + op_strin
            
        return strin
            
                         

In [22]:
tab = [[1,3,5], [2,4,6]]
tableau = Tableau(tab)

In [23]:
print(tableau.write_diagram())

1
1 3 5
2 4 6



In [24]:
print(tableau.ancestor(2).write_diagram())

[[1, 3, 5], [2, 4, 6]]
[[1, 3, 5], [2, 4]]
1
1 3
2 4



we can check that is it is standard or semistandard

In [25]:
tableau.is_standard()

True

In [26]:
tableau.is_semistandard()

True

In [40]:
tabs = [[[1,2,4], [3,5], [6]],
        [[1,2,4], [3,6], [5]],
        [[1,2,5], [3,4], [6]],
        [[1,2,5], [3,6], [4]],
        [[1,2,6], [3,4], [5]],
        [[1,2,6], [3,5], [4]]]


s = Operator([Symmetriser([4,5]), Symmetriser([1,2]), 
               Symmetriser([4,6], antisym = True), Symmetriser([1,3], antisym = True),
               Symmetriser([4,5]), Symmetriser([1,2])])

# n = 4 case
#tabs = [[[1,2], [3], [4]],
#        [[1,4], [2], [3]],
#        [[1,3], [2], [4]]]

#s = Operator([Symmetriser([1,2]), Symmetriser([3,4], antisym = True)])


molds1 = []
tabls = []
opers = {}

for tab in tabs:
    print('====================================')        
    tableau = Tableau(tab)
    tabls.append(tableau)
    
    mold_op = tableau.as_mold_operator()
    print(mold_op)
    print(tableau.write_diagram())
    molds1.append(mold_op)
    print([[op.domain, op.is_antisym()] for op in mold_op.operator_seq])
    oper = s.deepact_on(mold_op.deepact_on(s))
    print(oper.write_as_cycles())
    opers[tableau.write_diagram()] = oper
    print(opers.keys())

Operator has been normalised.
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
[[1, 2], [3]]
[[1, 2, 4], [3, 5], [6]]
[[1, 2, 4], [3, 5]]
[[1, 2, 4], [3]]
1
1 2 4
3 5
6

[[[1, 2], False], [[1, 3], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3, 6], True], [[2, 5], True], [[1, 2, 4], False], [[3, 5], False], [[1, 3], True], [[1, 2], False]]
+945()+945(1 2)+945(4 5)+945(1 2)(4 5)-945/2(1 3)-945/2(1 3 2)-945/2(1 3)(4 5)-945/2(1 3 2)(4 5)-270(4 6)-270(1 2)(4 6)-270(4 6 5)-270(1 2)(4 6 5)+135(1 3)(4 6)+135(1 3 2)(4 6)+135(1 3)(4 6 5)+135(1 3 2)(4 6 5)-945/2(1 2 3)-945/2(2 3)-945/2(1 2 3)(4 5)-945/2(2 3)(4 5)+135(1 2 3)(4 6)+135(2 3)(4 6)+135(1 2 3)(4 6 5)+135(2 3)(4 6 5)-675(4 5 6)-675(1 2)(4 5 6)-675(5 6)-675(1 2)(5 6)+675/2(1 3)(4 5 6)+675/2(1 3 2)(4 5 6)+675/2(1 3)(5 6)+675/2(1 3 2)(5 6)+675/2(1 2 3)(4 5 6)+675/2(2 3)(4 5 6)+675/2(1 2 3)(5

+405()+405(1 2)+405(4 5)+405(1 2)(4 5)-405/2(1 3)-405/2(1 3 2)-405/2(1 3)(4 5)-405/2(1 3 2)(4 5)-540(4 6)-540(1 2)(4 6)-540(4 6 5)-540(1 2)(4 6 5)+270(1 3)(4 6)+270(1 3 2)(4 6)+270(1 3)(4 6 5)+270(1 3 2)(4 6 5)-405/2(1 2 3)-405/2(2 3)-405/2(1 2 3)(4 5)-405/2(2 3)(4 5)+270(1 2 3)(4 6)+270(2 3)(4 6)+270(1 2 3)(4 6 5)+270(2 3)(4 6 5)+135(4 5 6)+135(1 2)(4 5 6)+135(5 6)+135(1 2)(5 6)-135/2(1 3)(4 5 6)-135/2(1 3 2)(4 5 6)-135/2(1 3)(5 6)-135/2(1 3 2)(5 6)-135/2(1 2 3)(4 5 6)-135/2(2 3)(4 5 6)-135/2(1 2 3)(5 6)-135/2(2 3)(5 6)+675/2(2 4)+675/2(1 2 4)+675/2(2 5 4)+675/2(1 2 5 4)-270(1 3)(2 4)-270(1 3 2 4)-270(1 3)(2 5 4)-270(1 3 2 5 4)-450(2 6 4)-450(1 2 6 4)-450(2 6 5 4)-450(1 2 6 5 4)+360(1 3)(2 6 4)+360(1 3 2 6 4)+360(1 3)(2 6 5 4)+360(1 3 2 6 5 4)-135/2(1 2 4 3)-135/2(2 4 3)-135/2(1 2 5 4 3)-135/2(2 5 4 3)+90(1 2 6 4 3)+90(2 6 4 3)+90(1 2 6 5 4 3)+90(2 6 5 4 3)+225/2(2 5 6 4)+225/2(1 2 5 6 4)+225/2(2 4)(5 6)+225/2(1 2 4)(5 6)-90(1 3)(2 5 6 4)-90(1 3 2 5 6 4)-90(1 3)(2 4)(5 6)-90(1 3 2 4)(

+756()+756(1 2)+756(4 5)+756(1 2)(4 5)-378(1 3)-378(1 3 2)-378(1 3)(4 5)-378(1 3 2)(4 5)-432(4 6)-432(1 2)(4 6)-432(4 6 5)-432(1 2)(4 6 5)+216(1 3)(4 6)+216(1 3 2)(4 6)+216(1 3)(4 6 5)+216(1 3 2)(4 6 5)-378(1 2 3)-378(2 3)-378(1 2 3)(4 5)-378(2 3)(4 5)+216(1 2 3)(4 6)+216(2 3)(4 6)+216(1 2 3)(4 6 5)+216(2 3)(4 6 5)-324(4 5 6)-324(1 2)(4 5 6)-324(5 6)-324(1 2)(5 6)+162(1 3)(4 5 6)+162(1 3 2)(4 5 6)+162(1 3)(5 6)+162(1 3 2)(5 6)+162(1 2 3)(4 5 6)+162(2 3)(4 5 6)+162(1 2 3)(5 6)+162(2 3)(5 6)+756(3 4)+756(1 2)(3 4)+756(3 5 4)+756(1 2)(3 5 4)-378(1 3 4)-378(1 3 4 2)-378(1 3 5 4)-378(1 3 5 4 2)-432(3 6 4)-432(1 2)(3 6 4)-432(3 6 5 4)-432(1 2)(3 6 5 4)+216(1 3 6 4)+216(1 3 6 4 2)+216(1 3 6 5 4)+216(1 3 6 5 4 2)-378(1 2 3 4)-378(2 3 4)-378(1 2 3 5 4)-378(2 3 5 4)+216(1 2 3 6 4)+216(2 3 6 4)+216(1 2 3 6 5 4)+216(2 3 6 5 4)-324(3 5 6 4)-324(1 2)(3 5 6 4)-324(3 4)(5 6)-324(1 2)(3 4)(5 6)+162(1 3 5 6 4)+162(1 3 5 6 4 2)+162(1 3 4)(5 6)+162(1 3 4 2)(5 6)+162(1 2 3 5 6 4)+162(2 3 5 6 4)+162(1 2 3 4

+225()+225(1 2)+225(4 5)+225(1 2)(4 5)-225/2(1 3)-225/2(1 3 2)-225/2(1 3)(4 5)-225/2(1 3 2)(4 5)-225/2(1 2 3)-225/2(2 3)-225/2(1 2 3)(4 5)-225/2(2 3)(4 5)-225(4 5 6)-225(1 2)(4 5 6)-225(5 6)-225(1 2)(5 6)+225/2(1 3)(4 5 6)+225/2(1 3 2)(4 5 6)+225/2(1 3)(5 6)+225/2(1 3 2)(5 6)+225/2(1 2 3)(4 5 6)+225/2(2 3)(4 5 6)+225/2(1 2 3)(5 6)+225/2(2 3)(5 6)-225(3 4)-225(1 2)(3 4)-225(3 5 4)-225(1 2)(3 5 4)+225/2(1 3 4)+225/2(1 3 4 2)+225/2(1 3 5 4)+225/2(1 3 5 4 2)+225/2(1 2 3 4)+225/2(2 3 4)+225/2(1 2 3 5 4)+225/2(2 3 5 4)+225(3 5 6 4)+225(1 2)(3 5 6 4)+225(3 4)(5 6)+225(1 2)(3 4)(5 6)-225/2(1 3 5 6 4)-225/2(1 3 5 6 4 2)-225/2(1 3 4)(5 6)-225/2(1 3 4 2)(5 6)-225/2(1 2 3 5 6 4)-225/2(2 3 5 6 4)-225/2(1 2 3 4)(5 6)-225/2(2 3 4)(5 6)+225/2(1 4 3)+225/2(1 4 3 2)+225/2(1 5 4 3)+225/2(1 5 4 3 2)-225/2(1 4)-225/2(1 4 2)-225/2(1 5 4)-225/2(1 5 4 2)-225/2(1 5 6 4 3)-225/2(1 5 6 4 3 2)-225/2(1 4 3)(5 6)-225/2(1 4 3 2)(5 6)+225/2(1 5 6 4)+225/2(1 5 6 4 2)+225/2(1 4)(5 6)+225/2(1 4 2)(5 6)+150(3 6)+150(1 2)

+9()+9(1 2)+9(4 5)+9(1 2)(4 5)-9/2(1 3)-9/2(1 3 2)-9/2(1 3)(4 5)-9/2(1 3 2)(4 5)-9/2(1 2 3)-9/2(2 3)-9/2(1 2 3)(4 5)-9/2(2 3)(4 5)-9(4 5 6)-9(1 2)(4 5 6)-9(5 6)-9(1 2)(5 6)+9/2(1 3)(4 5 6)+9/2(1 3 2)(4 5 6)+9/2(1 3)(5 6)+9/2(1 3 2)(5 6)+9/2(1 2 3)(4 5 6)+9/2(2 3)(4 5 6)+9/2(1 2 3)(5 6)+9/2(2 3)(5 6)+9(3 4)+9(1 2)(3 4)+9(3 5 4)+9(1 2)(3 5 4)-9/2(1 3 4)-9/2(1 3 4 2)-9/2(1 3 5 4)-9/2(1 3 5 4 2)-9/2(1 2 3 4)-9/2(2 3 4)-9/2(1 2 3 5 4)-9/2(2 3 5 4)-9(3 5 6 4)-9(1 2)(3 5 6 4)-9(3 4)(5 6)-9(1 2)(3 4)(5 6)+9/2(1 3 5 6 4)+9/2(1 3 5 6 4 2)+9/2(1 3 4)(5 6)+9/2(1 3 4 2)(5 6)+9/2(1 2 3 5 6 4)+9/2(2 3 5 6 4)+9/2(1 2 3 4)(5 6)+9/2(2 3 4)(5 6)-9/2(2 4)-9/2(1 2 4)-9/2(2 5 4)-9/2(1 2 5 4)+9(1 3)(2 4)+9(1 3 2 4)+9(1 3)(2 5 4)+9(1 3 2 5 4)-9/2(1 2 4 3)-9/2(2 4 3)-9/2(1 2 5 4 3)-9/2(2 5 4 3)+9/2(2 5 6 4)+9/2(1 2 5 6 4)+9/2(2 4)(5 6)+9/2(1 2 4)(5 6)-9(1 3)(2 5 6 4)-9(1 3 2 5 6 4)-9(1 3)(2 4)(5 6)-9(1 3 2 4)(5 6)+9/2(1 2 5 6 4 3)+9/2(2 5 6 4 3)+9/2(1 2 4 3)(5 6)+9/2(2 4 3)(5 6)-18(3 5)-18(1 2)(3 5)-18(3 4 5)-

+405()+405(1 2)+405(4 5)+405(1 2)(4 5)-405/2(1 3)-405/2(1 3 2)-405/2(1 3)(4 5)-405/2(1 3 2)(4 5)-270(4 6)-270(1 2)(4 6)-270(4 6 5)-270(1 2)(4 6 5)+135(1 3)(4 6)+135(1 3 2)(4 6)+135(1 3)(4 6 5)+135(1 3 2)(4 6 5)-405/2(1 2 3)-405/2(2 3)-405/2(1 2 3)(4 5)-405/2(2 3)(4 5)+135(1 2 3)(4 6)+135(2 3)(4 6)+135(1 2 3)(4 6 5)+135(2 3)(4 6 5)-135(4 5 6)-135(1 2)(4 5 6)-135(5 6)-135(1 2)(5 6)+135/2(1 3)(4 5 6)+135/2(1 3 2)(4 5 6)+135/2(1 3)(5 6)+135/2(1 3 2)(5 6)+135/2(1 2 3)(4 5 6)+135/2(2 3)(4 5 6)+135/2(1 2 3)(5 6)+135/2(2 3)(5 6)-405/2(2 5)-405/2(1 2 5)-405/2(2 4 5)-405/2(1 2 4 5)+405/2(1 3)(2 5)+405/2(1 3 2 5)+405/2(1 3)(2 4 5)+405/2(1 3 2 4 5)+135(2 5)(4 6)+135(1 2 5)(4 6)+135(2 4 6 5)+135(1 2 4 6 5)-405/2(1 3)(2 5)(4 6)-405/2(1 3 2 5)(4 6)-405/2(1 3)(2 4 6 5)-405/2(1 3 2 4 6 5)+135/2(1 2 5 3)(4 6)+135/2(2 5 3)(4 6)+135/2(1 2 4 6 5 3)+135/2(2 4 6 5 3)+135/2(2 6 4 5)+135/2(1 2 6 4 5)+135/2(2 6 5)+135/2(1 2 6 5)-135/2(1 2 6 4 5 3)-135/2(2 6 4 5 3)-135/2(1 2 6 5 3)-135/2(2 6 5 3)-405(3 4)-405(1 

In [41]:
print(' ')
print('=====================================================================')
print('=====================================================================')
print(' ')
for op in opers.keys():
    for otherop in opers.keys():
        print(op)
        op1 = cp.deepcopy(opers[op])
        fac1 = op1.get_overall_factor()
        print(fac1)
        print('and')
        print(otherop)
        op2 = cp.deepcopy(opers[otherop])
        fac2 = op2.get_overall_factor()
        print(fac2)
        
        if op1.is_equivalent_to(op2, up_to_factor = True):
            print('equiv')
        else:
            print('non equiv')
        
        print('------------------')

 
 
1
1 2 4
3 5
6

45
and
1
1 2 4
3 5
6

45
equiv
------------------
1
1 2 4
3 5
6

45
and
1
1 2 4
3 6
5

45
non equiv
------------------
1
1 2 4
3 5
6

45
and
1
1 2 5
3 4
6

18
non equiv
------------------
1
1 2 4
3 5
6

45
and
1
1 2 5
3 6
4

75
non equiv
------------------
1
1 2 4
3 5
6

45
and
1
1 2 6
3 4
5

3
non equiv
------------------
1
1 2 4
3 5
6

45
and
1
1 2 6
3 5
4

135
non equiv
------------------
1
1 2 4
3 6
5

45
and
1
1 2 4
3 5
6

45
non equiv
------------------
1
1 2 4
3 6
5

45
and
1
1 2 4
3 6
5

45
equiv
------------------
1
1 2 4
3 6
5

45
and
1
1 2 5
3 4
6

18
non equiv
------------------
1
1 2 4
3 6
5

45
and
1
1 2 5
3 6
4

75
non equiv
------------------
1
1 2 4
3 6
5

45
and
1
1 2 6
3 4
5

3
non equiv
------------------
1
1 2 4
3 6
5

45
and
1
1 2 6
3 5
4

135
non equiv
------------------
1
1 2 5
3 4
6

18
and
1
1 2 4
3 5
6

45
non equiv
------------------
1
1 2 5
3 4
6

18
and
1
1 2 4
3 6
5

45
non equiv
------------------
1
1 2 5
3 4
6

18
and
1
1 2 5
3 4
6

1

In [43]:
print(' ')
print('=====================================================================')
print('=====================================================================')
print(' ')
mults = {}
for op in opers.keys():
    for otherop in opers.keys():
        print(op)
        op1 = cp.deepcopy(opers[op])
        fac1 = op1.get_overall_factor()
        print(fac1)
        print('and')
        print(otherop)
        op2 = cp.deepcopy(opers[otherop])
        fac2 = op2.get_overall_factor()
        print(fac2)
        
        mult = op1.act_on(op2)
        mults[op+otherop] = mult
        print(mult.write_as_cycles())
        
        print('------------------')

 
 
1
1 2 4
3 5
6

45
and
1
1 2 4
3 5
6

45
+40186125()+40186125(1 2)+40186125(4 5)+40186125(1 2)(4 5)-40186125/2(1 3)-40186125/2(1 3 2)-40186125/2(1 3)(4 5)-40186125/2(1 3 2)(4 5)-11481750(4 6)-11481750(1 2)(4 6)-11481750(4 6 5)-11481750(1 2)(4 6 5)+5740875(1 3)(4 6)+5740875(1 3 2)(4 6)+5740875(1 3)(4 6 5)+5740875(1 3 2)(4 6 5)-40186125/2(1 2 3)-40186125/2(2 3)-40186125/2(1 2 3)(4 5)-40186125/2(2 3)(4 5)+5740875(1 2 3)(4 6)+5740875(2 3)(4 6)+5740875(1 2 3)(4 6 5)+5740875(2 3)(4 6 5)-28704375(4 5 6)-28704375(1 2)(4 5 6)-28704375(5 6)-28704375(1 2)(5 6)+28704375/2(1 3)(4 5 6)+28704375/2(1 3 2)(4 5 6)+28704375/2(1 3)(5 6)+28704375/2(1 3 2)(5 6)+28704375/2(1 2 3)(4 5 6)+28704375/2(2 3)(4 5 6)+28704375/2(1 2 3)(5 6)+28704375/2(2 3)(5 6)+53581500(3 5)+53581500(1 2)(3 5)+53581500(3 4 5)+53581500(1 2)(3 4 5)-26790750(1 3 5)-26790750(1 3 5 2)-26790750(1 3 4 5)-26790750(1 3 4 5 2)-21049875(3 5)(4 6)-21049875(1 2)(3 5)(4 6)-21049875(3 4 6 5)-21049875(1 2)(3 4 6 5)+21049875/2(1 3 5)(4 6)+21049875

+820125()+820125(1 2)+820125(4 5)+820125(1 2)(4 5)-820125/2(1 3)-820125/2(1 3 2)-820125/2(1 3)(4 5)-820125/2(1 3 2)(4 5)+1640250(4 6)+1640250(1 2)(4 6)+1640250(4 6 5)+1640250(1 2)(4 6 5)-820125(1 3)(4 6)-820125(1 3 2)(4 6)-820125(1 3)(4 6 5)-820125(1 3 2)(4 6 5)-820125/2(1 2 3)-820125/2(2 3)-820125/2(1 2 3)(4 5)-820125/2(2 3)(4 5)-820125(1 2 3)(4 6)-820125(2 3)(4 6)-820125(1 2 3)(4 6 5)-820125(2 3)(4 6 5)-2460375(4 5 6)-2460375(1 2)(4 5 6)-2460375(5 6)-2460375(1 2)(5 6)+2460375/2(1 3)(4 5 6)+2460375/2(1 3 2)(4 5 6)+2460375/2(1 3)(5 6)+2460375/2(1 3 2)(5 6)+2460375/2(1 2 3)(4 5 6)+2460375/2(2 3)(4 5 6)+2460375/2(1 2 3)(5 6)+2460375/2(2 3)(5 6)+1366875/2(2 4)+1366875/2(1 2 4)+1366875/2(2 5 4)+1366875/2(1 2 5 4)-546750(1 3)(2 4)-546750(1 3 2 4)-546750(1 3)(2 5 4)-546750(1 3 2 5 4)+1366875(2 6 4)+1366875(1 2 6 4)+1366875(2 6 5 4)+1366875(1 2 6 5 4)-1093500(1 3)(2 6 4)-1093500(1 3 2 6 4)-1093500(1 3)(2 6 5 4)-1093500(1 3 2 6 5 4)-273375/2(1 2 4 3)-273375/2(2 4 3)-273375/2(1 2 5 4 3)-273375/

+32148900()+32148900(1 2)+32148900(4 5)+32148900(1 2)(4 5)-16074450(1 3)-16074450(1 3 2)-16074450(1 3)(4 5)-16074450(1 3 2)(4 5)-18370800(4 6)-18370800(1 2)(4 6)-18370800(4 6 5)-18370800(1 2)(4 6 5)+9185400(1 3)(4 6)+9185400(1 3 2)(4 6)+9185400(1 3)(4 6 5)+9185400(1 3 2)(4 6 5)-16074450(1 2 3)-16074450(2 3)-16074450(1 2 3)(4 5)-16074450(2 3)(4 5)+9185400(1 2 3)(4 6)+9185400(2 3)(4 6)+9185400(1 2 3)(4 6 5)+9185400(2 3)(4 6 5)-13778100(4 5 6)-13778100(1 2)(4 5 6)-13778100(5 6)-13778100(1 2)(5 6)+6889050(1 3)(4 5 6)+6889050(1 3 2)(4 5 6)+6889050(1 3)(5 6)+6889050(1 3 2)(5 6)+6889050(1 2 3)(4 5 6)+6889050(2 3)(4 5 6)+6889050(1 2 3)(5 6)+6889050(2 3)(5 6)+32148900(3 4)+32148900(1 2)(3 4)+32148900(3 5 4)+32148900(1 2)(3 5 4)-16074450(1 3 4)-16074450(1 3 4 2)-16074450(1 3 5 4)-16074450(1 3 5 4 2)-18370800(3 6 4)-18370800(1 2)(3 6 4)-18370800(3 6 5 4)-18370800(1 2)(3 6 5 4)+9185400(1 3 6 4)+9185400(1 3 6 4 2)+9185400(1 3 6 5 4)+9185400(1 3 6 5 4 2)-16074450(1 2 3 4)-16074450(2 3 4)-16074450(1 

+455625()+455625(1 2)+455625(4 5)+455625(1 2)(4 5)-455625/2(1 3)-455625/2(1 3 2)-455625/2(1 3)(4 5)-455625/2(1 3 2)(4 5)-455625/2(1 2 3)-455625/2(2 3)-455625/2(1 2 3)(4 5)-455625/2(2 3)(4 5)+455625(4 5 6)+455625(1 2)(4 5 6)+455625(5 6)+455625(1 2)(5 6)-455625/2(1 3)(4 5 6)-455625/2(1 3 2)(4 5 6)-455625/2(1 3)(5 6)-455625/2(1 3 2)(5 6)-455625/2(1 2 3)(4 5 6)-455625/2(2 3)(4 5 6)-455625/2(1 2 3)(5 6)-455625/2(2 3)(5 6)-455625(3 4)-455625(1 2)(3 4)-455625(3 5 4)-455625(1 2)(3 5 4)+455625/2(1 3 4)+455625/2(1 3 4 2)+455625/2(1 3 5 4)+455625/2(1 3 5 4 2)+455625/2(1 2 3 4)+455625/2(2 3 4)+455625/2(1 2 3 5 4)+455625/2(2 3 5 4)-455625(3 5 6 4)-455625(1 2)(3 5 6 4)-455625(3 4)(5 6)-455625(1 2)(3 4)(5 6)+455625/2(1 3 5 6 4)+455625/2(1 3 5 6 4 2)+455625/2(1 3 4)(5 6)+455625/2(1 3 4 2)(5 6)+455625/2(1 2 3 5 6 4)+455625/2(2 3 5 6 4)+455625/2(1 2 3 4)(5 6)+455625/2(2 3 4)(5 6)+455625/2(1 4 3)+455625/2(1 4 3 2)+455625/2(1 5 4 3)+455625/2(1 5 4 3 2)-455625/2(1 4)-455625/2(1 4 2)-455625/2(1 5 4)-455625/

+91125()+91125(1 2)+91125(4 5)+91125(1 2)(4 5)-91125/2(1 3)-91125/2(1 3 2)-91125/2(1 3)(4 5)-91125/2(1 3 2)(4 5)-91125/2(1 2 3)-91125/2(2 3)-91125/2(1 2 3)(4 5)-91125/2(2 3)(4 5)-455625(4 5 6)-455625(1 2)(4 5 6)-455625(5 6)-455625(1 2)(5 6)+455625/2(1 3)(4 5 6)+455625/2(1 3 2)(4 5 6)+455625/2(1 3)(5 6)+455625/2(1 3 2)(5 6)+455625/2(1 2 3)(4 5 6)+455625/2(2 3)(4 5 6)+455625/2(1 2 3)(5 6)+455625/2(2 3)(5 6)+91125(3 4)+91125(1 2)(3 4)+91125(3 5 4)+91125(1 2)(3 5 4)-91125/2(1 3 4)-91125/2(1 3 4 2)-91125/2(1 3 5 4)-91125/2(1 3 5 4 2)-91125/2(1 2 3 4)-91125/2(2 3 4)-91125/2(1 2 3 5 4)-91125/2(2 3 5 4)-455625(3 5 6 4)-455625(1 2)(3 5 6 4)-455625(3 4)(5 6)-455625(1 2)(3 4)(5 6)+455625/2(1 3 5 6 4)+455625/2(1 3 5 6 4 2)+455625/2(1 3 4)(5 6)+455625/2(1 3 4 2)(5 6)+455625/2(1 2 3 5 6 4)+455625/2(2 3 5 6 4)+455625/2(1 2 3 4)(5 6)+455625/2(2 3 4)(5 6)-91125/2(2 4)-91125/2(1 2 4)-91125/2(2 5 4)-91125/2(1 2 5 4)+91125(1 3)(2 4)+91125(1 3 2 4)+91125(1 3)(2 5 4)+91125(1 3 2 5 4)-91125/2(1 2 4 3)-91125/

...

+4100625()+4100625(1 2)+4100625(4 5)+4100625(1 2)(4 5)-4100625/2(1 3)-4100625/2(1 3 2)-4100625/2(1 3)(4 5)-4100625/2(1 3 2)(4 5)-8201250(4 6)-8201250(1 2)(4 6)-8201250(4 6 5)-8201250(1 2)(4 6 5)+4100625(1 3)(4 6)+4100625(1 3 2)(4 6)+4100625(1 3)(4 6 5)+4100625(1 3 2)(4 6 5)-4100625/2(1 2 3)-4100625/2(2 3)-4100625/2(1 2 3)(4 5)-4100625/2(2 3)(4 5)+4100625(1 2 3)(4 6)+4100625(2 3)(4 6)+4100625(1 2 3)(4 6 5)+4100625(2 3)(4 6 5)+4100625(4 5 6)+4100625(1 2)(4 5 6)+4100625(5 6)+4100625(1 2)(5 6)-4100625/2(1 3)(4 5 6)-4100625/2(1 3 2)(4 5 6)-4100625/2(1 3)(5 6)-4100625/2(1 3 2)(5 6)-4100625/2(1 2 3)(4 5 6)-4100625/2(2 3)(4 5 6)-4100625/2(1 2 3)(5 6)-4100625/2(2 3)(5 6)+6834375/2(2 4)+6834375/2(1 2 4)+6834375/2(2 5 4)+6834375/2(1 2 5 4)-2733750(1 3)(2 4)-2733750(1 3 2 4)-2733750(1 3)(2 5 4)-2733750(1 3 2 5 4)-6834375(2 6 4)-6834375(1 2 6 4)-6834375(2 6 5 4)-6834375(1 2 6 5 4)+5467500(1 3)(2 6 4)+5467500(1 3 2 6 4)+5467500(1 3)(2 6 5 4)+5467500(1 3 2 6 5 4)-1366875/2(1 2 4 3)-1366875/2(2 4 3)-1

+3280500()+3280500(1 2)+3280500(4 5)+3280500(1 2)(4 5)-1640250(1 3)-1640250(1 3 2)-1640250(1 3)(4 5)-1640250(1 3 2)(4 5)-1640250(1 2 3)-1640250(2 3)-1640250(1 2 3)(4 5)-1640250(2 3)(4 5)-3280500(4 5 6)-3280500(1 2)(4 5 6)-3280500(5 6)-3280500(1 2)(5 6)+1640250(1 3)(4 5 6)+1640250(1 3 2)(4 5 6)+1640250(1 3)(5 6)+1640250(1 3 2)(5 6)+1640250(1 2 3)(4 5 6)+1640250(2 3)(4 5 6)+1640250(1 2 3)(5 6)+1640250(2 3)(5 6)+3280500(3 4)+3280500(1 2)(3 4)+3280500(3 5 4)+3280500(1 2)(3 5 4)-1640250(1 3 4)-1640250(1 3 4 2)-1640250(1 3 5 4)-1640250(1 3 5 4 2)-1640250(1 2 3 4)-1640250(2 3 4)-1640250(1 2 3 5 4)-1640250(2 3 5 4)-3280500(3 5 6 4)-3280500(1 2)(3 5 6 4)-3280500(3 4)(5 6)-3280500(1 2)(3 4)(5 6)+1640250(1 3 5 6 4)+1640250(1 3 5 6 4 2)+1640250(1 3 4)(5 6)+1640250(1 3 4 2)(5 6)+1640250(1 2 3 5 6 4)+1640250(2 3 5 6 4)+1640250(1 2 3 4)(5 6)+1640250(2 3 4)(5 6)-1640250(2 4)-1640250(1 2 4)-1640250(2 5 4)-1640250(1 2 5 4)+3280500(1 3)(2 4)+3280500(1 3 2 4)+3280500(1 3)(2 5 4)+3280500(1 3 2 5 4)-1640250

+2278125()+2278125(1 2)+2278125(4 5)+2278125(1 2)(4 5)-2278125/2(1 3)-2278125/2(1 3 2)-2278125/2(1 3)(4 5)-2278125/2(1 3 2)(4 5)-2278125/2(1 2 3)-2278125/2(2 3)-2278125/2(1 2 3)(4 5)-2278125/2(2 3)(4 5)-3189375(4 5 6)-3189375(1 2)(4 5 6)-3189375(5 6)-3189375(1 2)(5 6)+3189375/2(1 3)(4 5 6)+3189375/2(1 3 2)(4 5 6)+3189375/2(1 3)(5 6)+3189375/2(1 3 2)(5 6)+3189375/2(1 2 3)(4 5 6)+3189375/2(2 3)(4 5 6)+3189375/2(1 2 3)(5 6)+3189375/2(2 3)(5 6)-2278125(3 4)-2278125(1 2)(3 4)-2278125(3 5 4)-2278125(1 2)(3 5 4)+2278125/2(1 3 4)+2278125/2(1 3 4 2)+2278125/2(1 3 5 4)+2278125/2(1 3 5 4 2)+2278125/2(1 2 3 4)+2278125/2(2 3 4)+2278125/2(1 2 3 5 4)+2278125/2(2 3 5 4)+3189375(3 5 6 4)+3189375(1 2)(3 5 6 4)+3189375(3 4)(5 6)+3189375(1 2)(3 4)(5 6)-3189375/2(1 3 5 6 4)-3189375/2(1 3 5 6 4 2)-3189375/2(1 3 4)(5 6)-3189375/2(1 3 4 2)(5 6)-3189375/2(1 2 3 5 6 4)-3189375/2(2 3 5 6 4)-3189375/2(1 2 3 4)(5 6)-3189375/2(2 3 4)(5 6)+2278125/2(1 4 3)+2278125/2(1 4 3 2)+2278125/2(1 5 4 3)+2278125/2(1 5 4 3 2)-2

+164025()+164025(1 2)+164025(4 5)+164025(1 2)(4 5)-164025/2(1 3)-164025/2(1 3 2)-164025/2(1 3)(4 5)-164025/2(1 3 2)(4 5)-164025/2(1 2 3)-164025/2(2 3)-164025/2(1 2 3)(4 5)-164025/2(2 3)(4 5)-164025(4 5 6)-164025(1 2)(4 5 6)-164025(5 6)-164025(1 2)(5 6)+164025/2(1 3)(4 5 6)+164025/2(1 3 2)(4 5 6)+164025/2(1 3)(5 6)+164025/2(1 3 2)(5 6)+164025/2(1 2 3)(4 5 6)+164025/2(2 3)(4 5 6)+164025/2(1 2 3)(5 6)+164025/2(2 3)(5 6)+164025(3 4)+164025(1 2)(3 4)+164025(3 5 4)+164025(1 2)(3 5 4)-164025/2(1 3 4)-164025/2(1 3 4 2)-164025/2(1 3 5 4)-164025/2(1 3 5 4 2)-164025/2(1 2 3 4)-164025/2(2 3 4)-164025/2(1 2 3 5 4)-164025/2(2 3 5 4)-164025(3 5 6 4)-164025(1 2)(3 5 6 4)-164025(3 4)(5 6)-164025(1 2)(3 4)(5 6)+164025/2(1 3 5 6 4)+164025/2(1 3 5 6 4 2)+164025/2(1 3 4)(5 6)+164025/2(1 3 4 2)(5 6)+164025/2(1 2 3 5 6 4)+164025/2(2 3 5 6 4)+164025/2(1 2 3 4)(5 6)+164025/2(2 3 4)(5 6)-164025/2(2 4)-164025/2(1 2 4)-164025/2(2 5 4)-164025/2(1 2 5 4)+164025(1 3)(2 4)+164025(1 3 2 4)+164025(1 3)(2 5 4)+164025(1 

+7381125()+7381125(1 2)+7381125(4 5)+7381125(1 2)(4 5)-7381125/2(1 3)-7381125/2(1 3 2)-7381125/2(1 3)(4 5)-7381125/2(1 3 2)(4 5)-4920750(4 6)-4920750(1 2)(4 6)-4920750(4 6 5)-4920750(1 2)(4 6 5)+2460375(1 3)(4 6)+2460375(1 3 2)(4 6)+2460375(1 3)(4 6 5)+2460375(1 3 2)(4 6 5)-7381125/2(1 2 3)-7381125/2(2 3)-7381125/2(1 2 3)(4 5)-7381125/2(2 3)(4 5)+2460375(1 2 3)(4 6)+2460375(2 3)(4 6)+2460375(1 2 3)(4 6 5)+2460375(2 3)(4 6 5)-2460375(4 5 6)-2460375(1 2)(4 5 6)-2460375(5 6)-2460375(1 2)(5 6)+2460375/2(1 3)(4 5 6)+2460375/2(1 3 2)(4 5 6)+2460375/2(1 3)(5 6)+2460375/2(1 3 2)(5 6)+2460375/2(1 2 3)(4 5 6)+2460375/2(2 3)(4 5 6)+2460375/2(1 2 3)(5 6)+2460375/2(2 3)(5 6)-7381125/2(2 5)-7381125/2(1 2 5)-7381125/2(2 4 5)-7381125/2(1 2 4 5)+7381125/2(1 3)(2 5)+7381125/2(1 3 2 5)+7381125/2(1 3)(2 4 5)+7381125/2(1 3 2 4 5)+2460375(2 5)(4 6)+2460375(1 2 5)(4 6)+2460375(2 4 6 5)+2460375(1 2 4 6 5)-7381125/2(1 3)(2 5)(4 6)-7381125/2(1 3 2 5)(4 6)-7381125/2(1 3)(2 4 6 5)-7381125/2(1 3 2 4 6 5)+2460375/2

In [46]:
print(' ')
print('=====================================================================')
print('=====================================================================')
print(' ')

for op in mults.keys():
    for mult in mults.keys():
        print(op)
        op1 = cp.deepcopy(mults[op])
        fac1 = op1.get_overall_factor()
        print(fac1)
        print('and')
        print(mult)
        op2 = cp.deepcopy(mults[mult])
        fac2 = op2.get_overall_factor()
        print(fac2)
        
        print(op2.is_equivalent_to(op1, up_to_factor = True))
        
        print('------------------')

 
 
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
True
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 5
6
1
1 2 4
3 6
5

273375
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 5
6
1
1 2 5
3 6
4

455625
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 5
6
1
1 2 6
3 5
4

4100625
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 6
5
1
1 2 4
3 6
5

820125
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 6
5
1
1 2 5
3 4
6

109350
False
------------------
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
and
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875


False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 4
3 6
5
1
1 2 5
3 4
6

109350
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 4
3 6
5
1
1 2 6
3 4
5

91125
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 4
3 6
5
1
1 2 6
3 5
4

4100625
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 5
3 4
6
1
1 2 4
3 5
6

765450
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 5
3 4
6
1
1 2 4
3 6
5

109350
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 5
3 4
6
1
1 2 5
3 4
6

612360
True
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 5
3 4
6
1
1 2 5
3 6
4

182250
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 5
3 4
6
1
1 2 6
3 4
5

36450
False
------------------
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
and
1
1 2 5
3 4
6
1
1 2 6
3 5
4

False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 5
3 6
4
1
1 2 4
3 5
6

151875
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 5
3 6
4
1
1 2 4
3 6
5

455625
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 5
3 6
4
1
1 2 5
3 4
6

60750
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 5
3 6
4
1
1 2 5
3 6
4

759375
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 5
3 6
4
1
1 2 6
3 4
5

50625
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 5
3 6
4
1
1 2 6
3 5
4

2278125
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 6
3 4
5
1
1 2 4
3 5
6

30375
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 6
3 4
5
1
1 2 5
3 4
6

12150
False
------------------
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
and
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
False

False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 4
5
1
1 2 6
3 4
5

1215
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 5
4
1
1 2 4
3 5
6

1366875
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 5
4
1
1 2 5
3 4
6

546750
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 5
4
1
1 2 5
3 6
4

455625
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
and
1
1 2 6
3 5
4
1
1 2 6
3 5
4

2460375
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 6
5

820125
and
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
False
------------------
1
1 2 4
3 6
5
1
1 2 4
3 6
5

820125
and
1
1 2 4
3 5
6
1
1 2 4
3 6


False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 5
6
1
1 2 5
3 6
4

455625
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 5
6
1
1 2 6
3 5
4

4100625
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 6
5
1
1 2 4
3 6
5

820125
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 6
5
1
1 2 5
3 4
6

109350
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
True
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 6
5
1
1 2 6
3 4
5

91125
False
------------------
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
and
1
1 2 4
3 6
5
1
1

...

False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 5
6
1
1 2 4
3 6
5

273375
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 5
6
1
1 2 5
3 6
4

455625
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 5
6
1
1 2 6
3 4
5

91125
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 5
6
1
1 2 6
3 5
4

4100625
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 6
5
1
1 2 4
3 6
5

820125
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 6
5
1
1 2 5
3 4
6

109350
False
------------------
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
and
1
1 2 4
3 6
5
1
1 2 5
3 6
4

136687

False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 4
3 6
5
1
1 2 6
3 5
4

4100625
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 4
6
1
1 2 4
3 5
6

765450
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 4
6
1
1 2 4
3 6
5

109350
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 4
6
1
1 2 5
3 4
6

612360
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 4
6
1
1 2 5
3 6
4

182250
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 4
6
1
1 2 6
3 4
5

36450
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 4
6
1
1 2 6
3 5
4

1640250
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 6
4
1
1 2 4
3 5
6

151875
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 6
4
1
1 2 4
3 6
5

455625
False
------------------
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
and
1
1 2 5
3 6
4
1
1 2 5
3 4
6

60750


False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 5
3 6
4
1
1 2 6
3 4
5

50625
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 5
3 6
4
1
1 2 6
3 5
4

2278125
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 4
5
1
1 2 4
3 5
6

30375
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 4
5
1
1 2 4
3 6
5

30375
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 4
5
1
1 2 5
3 4
6

12150
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 4
5
1
1 2 5
3 6
4

10125
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 4
5
1
1 2 6
3 4
5

1215
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
True
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 5
4
1
1 2 4
3 5
6

1366875
False
------------------
1
1 2 6
3 4
5
1
1 2 6
3 5
4

54675
and
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
False


False
------------------
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
and
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
True
------------------
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
and
1
1 2 6
3 5
4
1
1 2 5
3 4
6

546750
False
------------------
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
and
1
1 2 6
3 5
4
1
1 2 5
3 6
4

455625
False
------------------
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
and
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
False
------------------
1
1 2 6
3 5
4
1
1 2 4
3 6
5

1366875
and
1
1 2 6
3 5
4
1
1 2 6
3 5
4

2460375
False
------------------
1
1 2 6
3 5
4
1
1 2 5
3 4
6

546750
and
1
1 2 4
3 5
6
1
1 2 4
3 5
6

1913625
False
------------------
1
1 2 6
3 5
4
1
1 2 5
3 4
6

546750
and
1
1 2 4
3 5
6
1
1 2 4
3 6
5

273375
False
------------------
1
1 2 6
3 5
4
1
1 2 5
3 4
6

546750
and
1
1 2 4
3 5
6
1
1 2 5
3 4
6

765450
False
------------------
1
1 2 6
3 5
4
1
1 2 5
3 4
6

546750
and
1
1 2 4
3 5
6
1
1 2 5
3 6
4

455625
False
------------------
1
1 2 6
3 5
4
1
1 2 5
3 4
6

546750
and
1
1 2 4
3 5
6
1
1 2 

91125
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 4
3 5
6
1
1 2 6
3 5
4

4100625
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 4
3 6
5
1
1 2 4
3 5
6

273375
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 4
3 6
5
1
1 2 4
3 6
5

820125
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 4
3 6
5
1
1 2 5
3 4
6

109350
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 4
3 6
5
1
1 2 5
3 6
4

1366875
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 4
3 6
5
1
1 2 6
3 4
5

91125
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 4
3 6
5
1
1 2 6
3 5
4

4100625
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 5
3 4
6
1
1 2 4
3 5
6

765450
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 5
3 4
6
1
1 2 4
3 6
5

109350
False
------------------
1
1 2 6
3 5
4
1
1 2 6
3 4
5

54675
and
1
1 2 5
3 4
6
1
1 2 5
3 4
6


In [48]:
P124356 = Operator([Symmetriser([1,2,4]), Symmetriser([3,5]), 
           Symmetriser([1,3,6], antisym = True), Symmetriser([2,5], antisym = True),
           Symmetriser([1,2,4]), Symmetriser([3,5])])
P124365 = Operator([Symmetriser([1,3,5], antisym = True), Symmetriser([2,6], antisym = True),
                    Symmetriser([1,2,4]), Symmetriser([3,6]),
                    Symmetriser([1,3,5], antisym = True), Symmetriser([2,6], antisym = True)])
                    #Operator([Symmetriser([1,2,4]), Symmetriser([3,6]),
          #          Symmetriser([1,3,5], antisym = True), Symmetriser([2,6], antisym = True),
          #          Symmetriser([1,2,4]), Symmetriser([3,6])])

S = Operator([Symmetriser([4,5]), Symmetriser([1,2]), 
               Symmetriser([4,6], antisym = True), Symmetriser([1,3], antisym = True),
               Symmetriser([4,5]), Symmetriser([1,2])])
print(P124356.deepact_on(S.deepact_on(P124365)).write_as_cycles())


Operator has been normalised.
Operator has been normalised.
Operator has been normalised.
0


In [49]:
P124356_1 = Operator([Symmetriser([1,3,6], antisym = True), Symmetriser([2,5], antisym = True),
           Symmetriser([1,2,4]), Symmetriser([3,5]),
           Symmetriser([1,3,6], antisym = True), Symmetriser([2,5], antisym = True)], normalised = False)
P124365_1 = Operator([Symmetriser([1,2,4]), Symmetriser([3,6]),
                    Symmetriser([1,3,5], antisym = True), Symmetriser([2,6], antisym = True),
                    Symmetriser([1,2,4]), Symmetriser([3,6])], normalised = False)

print(P124356_1.deepact_on(S.deepact_on(P124365_1)).write_as_cycles())

0


In [56]:
P125346_1 = Operator([Symmetriser([1,3,6], antisym = True), Symmetriser([2,4], antisym = True),
           Symmetriser([1,2,5]), Symmetriser([3,4]),
           Symmetriser([1,3,6], antisym = True), Symmetriser([2,4], antisym = True)], normalised = False)
P125364_1 = Operator([Symmetriser([1,2,5]), Symmetriser([3,6]),
                    Symmetriser([1,3,4], antisym = True), Symmetriser([2,6], antisym = True),
                    Symmetriser([1,2,5]), Symmetriser([3,6])], normalised = False)

print(P125346_1.deepact_on(S.deepact_on(P125364_1)).write_as_cycles())

0


In [57]:
print(P124356_1.deepact_on(S.deepact_on(P125364_1)).write_as_cycles())

0


In [58]:
print(P124356_1.deepact_on(S.deepact_on(P125346_1)).write_as_cycles())

+1215/2()-1215/2(3 6)-1215/2(1 3)+1215/2(1 6 3)+1215/2(1 3 6)-1215/2(1 6)+1215/2(3 4)-405/2(2 4 3)-1215/2(3 4 6)+405/2(2 4 6 3)-1215/2(1 3 4)+405/2(1 3 2 4)+1215/2(1 6 3 4)-405/2(1 6 3 2 4)+1215/2(1 3 4 6)-405/2(1 3 2 4 6)-1215/2(1 6)(3 4)+405/2(1 6)(2 4 3)-1215/2(2 5)(3 4)-810(2 5 4 3)+1215/2(2 5)(3 4 6)+810(2 5 4 6 3)+1215/2(1 3 4)(2 5)+810(1 3 2 5 4)-1215/2(1 6 3 4)(2 5)-810(1 6 3 2 5 4)-1215/2(1 3 4 6)(2 5)-810(1 3 2 5 4 6)+1215/2(1 6)(2 5)(3 4)+810(1 6)(2 5 4 3)+1215/2(1 2)-1215/2(1 2)(3 6)-1215/2(1 2 3)+1215/2(1 2 6 3)+1215/2(1 2 3 6)-1215/2(1 2 6)+1215/2(1 2)(3 4)-405/2(1 4 3 2)-1215/2(1 2)(3 4 6)+405/2(1 4 6 3 2)-1215/2(1 2 3 4)+405/2(1 4)(2 3)+1215/2(1 2 6 3 4)-405/2(1 4)(2 6 3)+1215/2(1 2 3 4 6)-405/2(1 4 6)(2 3)-1215/2(1 2 6)(3 4)+405/2(1 4 3 2 6)-1215/2(1 5 2)(3 4)-810(1 5 4 3 2)+1215/2(1 5 2)(3 4 6)+810(1 5 4 6 3 2)+1215/2(1 5 2 3 4)+810(1 5 4)(2 3)-1215/2(1 5 2 6 3 4)-810(1 5 4)(2 6 3)-1215/2(1 5 2 3 4 6)-810(1 5 4 6)(2 3)+1215/2(1 5 2 6)(3 4)+810(1 5 4 3 2 6)-405(1 4 2 5

In [54]:
print(S.deepact_on(P124356_1.deepact_on(S)).is_equivalent_to(S.deepact_on(P124356.deepact_on(S)), up_to_factor = True))

False


In [55]:
print(P124356_1.deepact_on(S.deepact_on(P124356_1)).write_as_cycles())

+1215()-1215(2 5)-1215(3 6)+1215(2 5)(3 6)-1215(1 3)+1215(1 3)(2 5)+1215(1 6 3)-1215(1 6 3)(2 5)+1215(1 3 6)-1215(1 3 6)(2 5)-1215(1 6)+1215(1 6)(2 5)+1620(3 5)-1620(2 5 3)-1620(3 5 6)+1620(2 5 6 3)-1620(1 3 5)+1620(1 3 2 5)+1620(1 6 3 5)-1620(1 6 3 2 5)+1620(1 3 5 6)-1620(1 3 2 5 6)-1620(1 6)(3 5)+1620(1 6)(2 5 3)+405(2 4)(3 5)-405(2 4 5 3)-405(2 4)(3 5 6)+405(2 4 5 6 3)-405(1 3 5)(2 4)+405(1 3 2 4 5)+405(1 6 3 5)(2 4)-405(1 6 3 2 4 5)+405(1 3 5 6)(2 4)-405(1 3 2 4 5 6)-405(1 6)(2 4)(3 5)+405(1 6)(2 4 5 3)+1215(1 2)-1215(1 5 2)-1215(1 2)(3 6)+1215(1 5 2)(3 6)-1215(1 2 3)+1215(1 5 2 3)+1215(1 2 6 3)-1215(1 5 2 6 3)+1215(1 2 3 6)-1215(1 5 2 3 6)-1215(1 2 6)+1215(1 5 2 6)+1620(1 2)(3 5)-1620(1 5 3 2)-1620(1 2)(3 5 6)+1620(1 5 6 3 2)-1620(1 2 3 5)+1620(1 5)(2 3)+1620(1 2 6 3 5)-1620(1 5)(2 6 3)+1620(1 2 3 5 6)-1620(1 5 6)(2 3)-1620(1 2 6)(3 5)+1620(1 5 3 2 6)+405(1 4 2)(3 5)-405(1 4 5 3 2)-405(1 4 2)(3 5 6)+405(1 4 5 6 3 2)-405(1 4 2 3 5)+405(1 4 5)(2 3)+405(1 4 2 6 3 5)-405(1 4 5)(2 6 3)

In [61]:
op = tableau.as_operator()
print(op.is_normalised())
print(op.collapse().write_as_cycles())

#op.get('domain')

#print(op.write_latex())

NameError: name 'Operator' is not defined

In [39]:
coll = op.collapse()
coll.write_as_cycles()

NameError: name 'op' is not defined

In [20]:
lns = [1,2,3]
di = {str(num): [num, num] for num in lns}
list(di.keys())
len(lns)

3

In [21]:
np.arange(1,4)

array([1, 2, 3])

In [22]:
tab = [[1,2]]
mx = np.max([np.max(row) for row in tab])
print(mx)

print([[val for val in row if val != mx] for row in tab])

2
[[1]]


In [23]:
sum([1,1,0])

2

In [24]:
syms = [Symmetriser([1,2]), Symmetriser([3,4]), Symmetriser([1,3], antisym = True), Symmetriser([2,4], antisym = False), Symmetriser([1,2]), Symmetriser([3,4])]
op = Operator(syms)
print(op.is_idempotent())
print(op.is_normalised())
print(op.collapse().write_as_cycles())
print('pref', op.get('prefactor'))
op = op.collapse()
#print(op.get_overall_factor())

print(op.act_on(op).write_as_cycles())


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:244: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Operator has been normalised.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:246: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:244: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:254: DeprecationWarning: Fraction.__float__ returned non-float (type numpy.float64).  The ability to return an instance of a strict subclass of float is deprecated, and may be removed in a future version of Python.


True
True
+1/8()+1/8(3 4)+1/8(1 2)+1/8(1 2)(3 4)-1/8(1 3)(2 4)-1/8(1 3 2 4)-1/8(1 4 2 3)-1/8(1 4)(2 3)
pref 2
+1/8()+1/8(3 4)+1/8(1 2)+1/8(1 2)(3 4)-1/8(1 3)(2 4)-1/8(1 3 2 4)-1/8(1 4 2 3)-1/8(1 4)(2 3)


In [73]:
syms = [Symmetriser([1,2,4], normalise = False), Symmetriser([3,5], normalise = False), 
        Symmetriser([2,3,4], antisym = True, normalise = False), Symmetriser([1,2], normalise = False)] 
        #CycleSum([Cycle('(1,2,3,4,5,6)')])]
op = Operator(syms)
print(op.write_latex())

Operator has been normalised.
4
fir 0
domain [1, 2, 4]
----False---- at ind: 0
next op type: Falseat ind: 0
op_ind 0
new x 1
next op type: Falseat ind: 1
op_ind 0
new x 2
next op type: Trueat ind: 2
op_ind 2
new x 3
miss lin []
final cyc {'domain': array([1, 2, 3, 4, 5]), 'image': [1, 2, 4, 3, 5]}
\draw[symmetrizer](-0.4,1.62)rectangle(-0.1,0.78);
\draw[symmetrizer](-0.4,0.72)rectangle(-0.1,0.18);

cycle {'domain': array([1, 2, 3, 4, 5]), 'image': [1, 2, 4, 3, 5]}
fir 2
domain [2, 3, 4]
----True---- at ind: 2
next op type: Trueat ind: 2
op_ind 2
new x 3
next op type: Falseat ind: 3
op_ind 3
new x 4
miss lin [1, 5]
final cyc {'domain': array([1, 2, 3, 4, 5]), 'image': [1, 2, 3, 4, 5]}
\draw[antisymmetrizer](-0.9,1.32)rectangle(-0.6,0.48);

cycle {'domain': array([1, 2, 3, 4, 5]), 'image': [1, 2, 3, 4, 5]}
fir 3
domain [1, 2]
----False---- at ind: 3
next op type: Falseat ind: 3
op_ind 3
new x 4
miss lin [3, 4, 5]
final cyc {'domain': array([1, 2, 3, 4, 5]), 'image': [1, 2, 3, 4, 5]}
\dra

In [65]:
dic = read_yaml('latex_config.yaml')
print(dic['start string'])

\begin{tikzpicture}[baseline={([yshift=-.5ex]current bounding \ box.center)},vertex/.style=anchor=base,yscale=0.025,xscale=0.025,shift={(0,-0.8)}}]\n \ \draw[invisible](0,-0.7)circle(0.01);\n \draw[invisible](0,0.7)circle(0.01);\n



In [27]:
op.categorise_operators()

['Symmetriser', 'Symmetriser', 'Symmetriser', 'Symmetriser']


[[<__main__.Symmetriser at 0x2a026a48b48>],

In [28]:
op_seq = op.operator_seq
op_types = [tu.get_type(str(type(op))) for op in op_seq]
print(op_types)

['Symmetriser', 'Symmetriser', 'Symmetriser', 'Symmetriser']


In [75]:
op = 'x'
if op in 'x':
    op = 'y'
elif op in 'y':
    print('ye')